In [ ]:
# dga_feature_extractor.py
import pandas as pd
import numpy as np
import re
import math
import zlib
import gzip
import string
from collections import Counter
from nltk.corpus import words
from nltk import download
from difflib import SequenceMatcher
from scipy.stats import entropy, kurtosis
from sklearn.preprocessing import normalize

# Download dictionary
download('words')
english_words = set(w.lower() for w in words.words())

VOWELS = set("aeiou")
CONSONANTS = set(string.ascii_lowercase) - VOWELS
POPULAR_TLDS = {".com", ".org", ".net", ".info", ".edu", ".gov"}
BAD_TLDS = {".xyz", ".top", ".club", ".work", ".click"}

# Example benign distribution (uniform)
BENIGN_DIST = np.ones(26) / 26

def shannon_entropy(s):
    p, _ = np.histogram(list(s), bins=range(257), density=True)
    p = p[p > 0]
    return -np.sum(p * np.log2(p))

def normalized_entropy(s):
    if not s: return 0
    return shannon_entropy(s) / math.log2(len(set(s)) + 1)

def compress_ratio(s):
    if not s: return 0
    return len(zlib.compress(s.encode())) / max(1, len(s))

def dict_word_features(domain):
    matches = [w for w in english_words if w in domain]
    longest = max((len(w) for w in matches), default=0)
    return len(matches), longest, longest / max(1,len(domain))

def sliding_dict_ratio(domain, window=4):
    n = len(domain)
    if n < window: return 0
    matches = 0
    for i in range(n - window + 1):
        if domain[i:i+window] in english_words:
            matches += 1
    return matches / (n - window + 1)

def vowel_consonant_alt(domain):
    domain = re.sub(r'[^a-z]', '', domain.lower())
    count = 0
    for i in range(1, len(domain)):
        if (domain[i] in VOWELS and domain[i-1] in CONSONANTS) or (domain[i] in CONSONANTS and domain[i-1] in VOWELS):
            count += 1
    return count

def char_gini(s):
    counts = Counter(s)
    N = sum(counts.values())
    if N == 0: return 0
    gini = 1 - sum((c/N)**2 for c in counts.values())
    return gini

def kl_divergence(s):
    counts = Counter([c for c in s if c.isalpha()])
    N = sum(counts.values())
    if N == 0: return 0
    dist = np.array([counts.get(chr(97+i),0)/N for i in range(26)])
    return entropy(dist, BENIGN_DIST)

def markov_chain_likelihood(domain):
    if len(domain) < 2: return 0
    transitions = {}
    for i in range(len(domain)-1):
        pair = (domain[i], domain[i+1])
        transitions[pair] = transitions.get(pair, 0) + 1
    total = sum(transitions.values())
    return sum(math.log((c/total)+1e-6) for c in transitions.values())

def autocorrelation_score(domain):
    if len(domain) < 2: return 0
    values = [ord(c) for c in domain]
    mean = np.mean(values)
    var = np.var(values)
    corr = sum((values[i]-mean)*(values[i+1]-mean) for i in range(len(values)-1)) / (var*(len(values)-1)+1e-6)
    return corr

def hyphen_word_match_ratio(domain):
    """Compute ratio of hyphen-separated parts that are valid dictionary words."""
    parts = domain.split("-")
    if not parts:
        return 0.0
    dict_matches = sum(1 for p in parts if p in english_words)
    return dict_matches / len(parts)

# === Extra Feature Functions ===
def renyi_entropy(s, alpha=2):
    """Rényi entropy of order alpha (default α=2)."""
    if not s:
        return 0
    counts = Counter(s)
    probs = np.array(list(counts.values())) / len(s)
    if alpha == 1:
        return -np.sum(probs * np.log2(probs))  # Shannon
    return 1 / (1 - alpha) * np.log2(np.sum(probs ** alpha))

# Load top domains (free list like Tranco or Cisco Umbrella top domains)
try:
    with open("top1k_domains.txt") as f:
        popular_domains = [line.strip().lower() for line in f.readlines()]
except FileNotFoundError:
    popular_domains = ["google.com","facebook.com","youtube.com","amazon.com","wikipedia.org"]  # fallback demo list

def min_levenshtein_to_popular(domain, topN=500):
    """Compute minimum normalized Levenshtein distance to topN popular domains."""
    domain = domain.lower()
    min_dist = 1.0
    for pd in popular_domains[:topN]:
        ratio = SequenceMatcher(None, domain, pd).ratio()
        dist = 1 - ratio  # 0 = identical, 1 = very different
        if dist < min_dist:
            min_dist = dist
    return min_dist

# Keyboard adjacency mapping
KEYBOARD_NEIGHBORS = {
    'q': "was", 'w': "qase", 'e': "wsdr", 'r': "edft", 't': "rfgy",
    'y': "tghu", 'u': "yhj", 'i': "ujk", 'o': "ikl", 'p': "ol",
    'a': "qwsz", 's': "qwedxza", 'd': "erfcxs", 'f': "rtgvcd", 'g': "tyhbvf",
    'h': "yujnbg", 'j': "uikmnh", 'k': "iolmj", 'l': "opk",
    'z': "asx", 'x': "zsdc", 'c': "xdfv", 'v': "cfgb", 'b': "vghn",
    'n': "bhjm", 'm': "njk"
}

def keyboard_distance_score(domain):
    """Average keyboard adjacency match score between consecutive characters."""
    domain = re.sub(r'[^a-z]', '', domain.lower())
    if len(domain) < 2:
        return 0
    score = 0
    for i in range(len(domain)-1):
        if domain[i+1] in KEYBOARD_NEIGHBORS.get(domain[i], ""):
            score += 1
    return score / (len(domain)-1)

# === Updated extract_features ===
def extract_features(domain):
    if not isinstance(domain, str):
        domain = str(domain)
    domain = domain.lower()
    name, _, tld = domain.rpartition(".")
    if not name:
        name = domain






#"Length","Consonant_Count","Unique_Chars","Max_Cons_Cluster","Dist_STD","Char_Gini","Char_Freq_Deviation","KL_divergence","Compression_ratio","Pronounceability","Bigram_Score","Trigram_Score","Markov_Chain_Likelihood","Bigram_Likelihood","Ngram_LM_Perplexity","Unique_Char_Ratio","Norm_Char_Freq_Var","Kolmogorov_Complexity","Renyi_Entropy","Min_Levenshtein_to_Popular","Keyboard_Distance_Score","Sliding_Word_Ratio"

    feats = {}



    feats["Length"] = len(domain)
    feats["Consonant_Count"] = sum(c in CONSONANTS for c in domain)
    feats["Unique_Chars"] = len(set(domain))
    feats["Max_Cons_Cluster"] = max((len(m.group()) for m in re.finditer(r'[bcdfghjklmnpqrstvwxyz]+', domain)), default=0)

    # Info-theoretic
    feats["Dist_STD"] = np.std([domain.count(c) for c in set(domain)])
    feats["Char_Gini"] = char_gini(domain)
    feats["Char_Freq_Deviation"] = np.std(list(Counter(domain).values()))
    feats["KL_divergence"] = kl_divergence(domain)
    feats["Compression_ratio"] = compress_ratio(domain)

    # Pronounceability
    feats["Pronounceability"] = sum(c in VOWELS for c in domain) / (sum(c in CONSONANTS for c in domain) + 1)

    # N-gram / LM
    feats["Bigram_Score"] = sum(1 for i in range(len(domain) - 1) if domain[i].isalpha() and domain[i + 1].isalpha())
    feats["Trigram_Score"] = sum(1 for i in range(len(domain) - 2) if domain[i].isalpha() and domain[i + 2].isalpha())
    feats["Markov_Chain_Likelihood"] = markov_chain_likelihood(domain)
    feats["Bigram_Likelihood"] = sum(1 for i in range(len(domain)-1) if domain[i].isalpha() and domain[i+1].isalpha())/max(1,len(domain))
    feats["Ngram_LM_Perplexity"] = math.exp(-markov_chain_likelihood(domain) / max(1, len(domain)))

    # Structural/pattern
    feats["Unique_Char_Ratio"] = len(set(domain)) / max(1, len(domain))
    feats["Norm_Char_Freq_Var"] = np.std(list(Counter(domain).values())) / max(1, len(domain))



    feats["Kolmogorov_Complexity"] = compress_ratio(domain)

   # === Advanced Features Added ===
    feats["Renyi_Entropy"] = renyi_entropy(domain, alpha=2)
    feats["Min_Levenshtein_to_Popular"] = min_levenshtein_to_popular(domain)
    feats["Keyboard_Distance_Score"] = keyboard_distance_score(domain)
    feats["Sliding_Word_Ratio"] = sliding_dict_ratio(domain)

    #feats["Max_Repeated_Char_Run"] = max( (len(m.group()) for m in re.finditer(r'(.)\1+', domain)), default=0)





    ##feats["Digit_Count"] = sum(c.isdigit() for c in domain)
    ##feats["Digit_Ratio"] = feats["Digit_Count"]/max(1,len(domain))
    ##feats["Max_Consec_Digits"] = max((len(m.group()) for m in re.finditer(r'\d+', domain)), default=0)
    ##feats["Vowel_Count"] = sum(c in VOWELS for c in domain)
    ##feats["Has_Hyphen"] = "-" in domain
    ##feats["Hyphen_Placement_Score"] = 1 if domain.startswith("-") or domain.endswith("-") else 0
    ##feats["Hyphen_Word_Match_Ratio"] = hyphen_word_match_ratio(domain)
    ##feats["Subdomain_Depth"] = domain.count(".")
    ##feats["Entropy"] = shannon_entropy(domain)
    #feats["Normalized_Entropy"] = normalized_entropy(domain)
    ##feats["Vowel_Entropy"] = shannon_entropy("".join([c for c in domain if c in VOWELS]))
    ##feats["Consonant_Entropy"] = shannon_entropy("".join([c for c in domain if c not in VOWELS and c.isalpha()]))
    #dwc, lwl, lwr = dict_word_features(domain)
    #feats["Dict_Word_Count"] = dwc
    #feats["Longest_Word_Len"] = lwl
    #feats["Longest_Word_Ratio"] = lwr
    #feats["Word_Match_Ratio"] = dwc/max(1,len(domain.split(".")))
    #feats["Sliding_Word_Ratio"] = sliding_dict_ratio(domain)
    ##feats["Vowel_Consonant_Alt"] = vowel_consonant_alt(domain)
    #feats["Lexical_Complexity"] = shannon_entropy(domain)*len(set(domain))/max(1,len(domain))
    #feats["Bigram_Likelihood"] = sum(1 for i in range(len(domain)-1) if domain[i].isalpha() and domain[i+1].isalpha())/max(1,len(domain))
    #feats["TLD_Common"] = 1 if f".{tld}" in POPULAR_TLDS else 0
    ##feats["TLD_Bad_Score"] = 1 if f".{tld}" in BAD_TLDS else 0
    ##feats["Popular_Domain"] = 0  # needs Alexa dataset
    ##feats["Alexa_popularity_bucket"] = -1
    #feats["Repeat_Chars"] = max((len(m.group()) for m in re.finditer(r'(.)\1+', domain)), default=0)
    ##feats["Max_Repeated_Char_Run"] = feats["Repeat_Chars"]
    ##feats["Digit_Letter_Alt_Count"] = sum((domain[i].isdigit() != domain[i+1].isdigit()) for i in range(len(domain)-1))
    #feats["Palindromic_Substring_Ratio"] = sum(domain[i:j]==domain[i:j][::-1] for i in range(len(domain)) for j in range(i+2,len(domain)+1))/max(1,len(domain))
    ##feats["First_Last_Char_Type"] = int(domain[0].isalpha()) + int(domain[-1].isalpha())
    ##feats["Shannon_Window_Entropy"] = np.mean([shannon_entropy(domain[i:i+4]) for i in range(len(domain)-3)]) if len(domain)>=4 else  shannon_entropy(domain)
    ##feats["Markov_Cross_Entropy"] = shannon_entropy(domain) -  markov_chain_likelihood(domain)
    ##feats["Permutation_Entropy"] = shannon_entropy(domain)/max(1,math.log2(len(domain)))
    #feats["RunLength_Entropy"] = shannon_entropy("".join([str(len(m.group())) for m in re.finditer(r'(.)\1*', domain)]))
    #feats["Dict_Word_Coverage"] = dwc/max(1,len(domain))
    #feats["Longest_Word_Suffix"] = max((len(w) for w in english_words if domain.endswith(w)), default=0)
    #feats["Word_Boundary_Count"] = domain.count("-")
    #feats["Syllable_Count"] = sum(domain.count(v) for v in VOWELS)
    #feats["TLD_Length"] = len(tld)
    #feats["TLD_Rarity_Score"] = 0 if f".{tld}" in POPULAR_TLDS else 1
    #feats["Autocorrelation_Score"] = autocorrelation_score(domain)
    #feats["Hurst_Exponent"] = 0.5  # placeholder

    return feats

# === Run on a dataset ===
df = pd.read_csv("dga_version5.csv")  # must have "domain" column
features = df["domain"].apply(extract_features)
feat_df = pd.DataFrame(list(features))
out = pd.concat([df, feat_df], axis=1)
out.to_csv("dga_18_version1.csv", index=False)
print("✅ Features saved to domains_with_features.csv")

[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Package words is already up-to-date!


KeyboardInterrupt: 

In [ ]:
from google.colab import files
df=pd.read_csv("dga_18_version1.csv")
df.describe()

,label,Length,Consonant_Count,Unique_Chars,Max_Cons_Cluster,Dist_STD,Char_Gini,Char_Freq_Deviation,KL_divergence,Compression_ratio,...,Markov_Chain_Likelihood,Bigram_Likelihood,Ngram_LM_Perplexity,Unique_Char_Ratio,Norm_Char_Freq_Var,Kolmogorov_Complexity,Renyi_Entropy,Min_Levenshtein_to_Popular,Keyboard_Distance_Score,Sliding_Word_Ratio
count,694173.0,694173.000000,694173.000000,694173.000000,694173.000000,694173.000000,694173.000000,694173.000000,694173.000000,694173.000000,...,694173.000000,694173.000000,694173.000000,694173.000000,694173.000000,694173.000000,694173.000000,694173.000000,694173.000000,694173.000000
mean,1.0,19.226092,12.200742,12.481366,4.576519,0.712628,0.897836,0.712628,0.943430,1.469141,...,-51.455950,0.819104,13.817479,0.691829,0.036880,1.469141,3.324323,0.618239,0.141644,0.055668
std,0.0,6.938378,4.305605,2.689910,2.643387,0.338331,0.023024,0.338331,0.231497,0.180496,...,24.607247,0.067656,4.933978,0.143928,0.012843,0.180496,0.304480,0.083928,0.094402,0.082787
min,1.0,8.000000,2.000000,4.000000,1.000000,0.000000,0.641975,0.000000,0.260340,0.885714,...,-138.520733,0.156250,2.875434,0.305556,0.000000,0.885714,1.481869,0.142857,0.000000,0.000000
25%,1.0,14.000000,9.000000,10.000000,3.000000,0.421325,0.887574,0.421325,0.769302,1.320000,...,-68.735314,0.769231,9.912027,0.566667,0.029215,1.320000,3.152952,0.567568,0.071429,0.000000
50%,1.0,18.000000,12.000000,12.000000,4.000000,0.655555,0.901308,0.655555,0.919725,1.444444,...,-48.164338,0.823529,12.802302,0.687500,0.036666,1.444444,3.340923,0.615385,0.133333,0.000000
75%,1.0,25.000000,15.000000,14.000000,6.000000,0.942809,0.914062,0.942809,1.094141,1.571429,...,-33.344173,0.880000,16.398649,0.812500,0.042855,1.571429,3.540568,0.666667,0.212121,0.117647
max,1.0,40.000000,27.000000,26.000000,16.000000,2.661453,0.956633,2.661453,2.302397,2.000000,...,-10.982275,0.925000,34.614674,1.000000,0.177778,2.000000,4.527247,0.913043,0.857143,0.625000


In [ ]:
import pandas as pd

def reorder_columns(input_csv, output_csv, desired_order):
    """
    Reorder the columns of a CSV file based on user-defined order.

    Parameters:
    - input_csv (str): Path to input CSV file.
    - output_csv (str): Path to output CSV file with reordered columns.
    - desired_order (list): List of column names in the desired order.
    """
    # Load dataset
    df = pd.read_csv(input_csv)

    # Check which desired columns exist
    available_columns = [col for col in desired_order if col in df.columns]

    # Add missing columns (if any were not in df)
    missing_columns = [col for col in desired_order if col not in df.columns]
    for col in missing_columns:
        df[col] = None  # Fill with None or default values

    # Reorder
    df = df[available_columns + [col for col in df.columns if col not in available_columns]]

    # Save output
    df.to_csv(output_csv, index=False)
    print(f"✅ Reordered CSV saved as {output_csv}")


# =====================
# Example usage
# =====================

# Suppose your dataset has 58 features + "domain" + "label"
input_csv = "dga_18_version1.csv"
output_csv = "dga_18_version2.csv"

# User-defined column order (just an example)
desired_order = [
     #"domain","Length","Consonant_Count","Unique_Chars","Max_Cons_Cluster","Dist_STD","Char_Gini","Char_Freq_Deviation","KL_divergence",#"Compression_ratio",
     #"Pronounceability",#"Bigram_Score",
     #"Trigram_Score","Markov_Chain_Likelihood","Bigram_Likelihood","Ngram_LM_Perplexity","Unique_Char_Ratio","Norm_Char_Freq_Var","label"




     "domain","Length","Consonant_Count","Unique_Chars","Max_Cons_Cluster","Dist_STD","Char_Gini","Char_Freq_Deviation","KL_divergence","Compression_ratio","Pronounceability","Bigram_Score","Trigram_Score","Markov_Chain_Likelihood","Bigram_Likelihood","Ngram_LM_Perplexity","Unique_Char_Ratio","Norm_Char_Freq_Var","Kolmogorov_Complexity","Renyi_Entropy","Min_Levenshtein_to_Popular","Keyboard_Distance_Score","Sliding_Word_Ratio","label"
     #"Kolmogorov_Complexity"
     # "domain","Length","Consonant_Count","Max_Cons_Cluster","Dist_STD","Char_Gini","Char_Freq_Deviation","KL_divergence","Compression_ratio","Bigram_Score","Trigram_Score","Markov_Chain_Likelihood","Ngram_LM_Perplexity","Unique_Char_Ratio","Norm_Char_Freq_Var","label"
    #Digit_Count", "label"
       #"domain","Length","Consonant_Count","Unique_Chars","Max_Cons_Cluster","Dist_STD","Char_Gini","Char_Freq_Deviation","KL_divergence","Pronounceability","Trigram_Score","Markov_Chain_Likelihood","Bigram_Likelihood","Ngram_LM_Perplexity","Unique_Char_Ratio","Norm_Char_Freq_Var","label"
    #"domain","Char_Freq_Deviation","Unique_Char_Ratio","label"
    #"domain","Length","Consonant_Count","Unique_Chars","Max_Cons_Cluster","Dist_STD","Char_Gini","Char_Freq_Deviation","KL_divergence","Compression_ratio","Pronounceability","Bigram_Score","Trigram_Score","Markov_Chain_Likelihood","Ngram_LM_Perplexity","Unique_Char_Ratio","Norm_Char_Freq_Var","label"
    #Digit_Count", "label",
    #"Length", "Digit_Ratio", "Max_Consec_Digits","Vowel_Count", , , "KL-divergence from benign char dist",
    #"Compression ratio", "Pronounceability", "Bigram_Score", "Trigram_Score",
    #"Alexa popularity bucket", "Homoglyph/brand-similarity (lookalike)"
]

# Reorder dataset
reorder_columns(input_csv, output_csv, desired_order)


✅ Reordered CSV saved as dga_18_version2.csv


In [ ]:
from google.colab import files
df=pd.read_csv("dga_18_version2.csv")
df.describe()

,Length,Consonant_Count,Unique_Chars,Max_Cons_Cluster,Dist_STD,Char_Gini,Char_Freq_Deviation,KL_divergence,Compression_ratio,Pronounceability,...,Bigram_Likelihood,Ngram_LM_Perplexity,Unique_Char_Ratio,Norm_Char_Freq_Var,Kolmogorov_Complexity,Renyi_Entropy,Min_Levenshtein_to_Popular,Keyboard_Distance_Score,Sliding_Word_Ratio,label
count,694173.000000,694173.000000,694173.000000,694173.000000,694173.000000,694173.000000,694173.000000,694173.000000,694173.000000,694173.000000,...,694173.000000,694173.000000,694173.000000,694173.000000,694173.000000,694173.000000,694173.000000,694173.000000,694173.000000,694173.0
mean,19.226092,12.200742,12.481366,4.576519,0.712628,0.897836,0.712628,0.943430,1.469141,0.466916,...,0.819104,13.817479,0.691829,0.036880,1.469141,3.324323,0.618239,0.141644,0.055668,1.0
std,6.938378,4.305605,2.689910,2.643387,0.338331,0.023024,0.338331,0.231497,0.180496,0.246663,...,0.067656,4.933978,0.143928,0.012843,0.180496,0.304480,0.083928,0.094402,0.082787,0.0
min,8.000000,2.000000,4.000000,1.000000,0.000000,0.641975,0.000000,0.260340,0.885714,0.000000,...,0.156250,2.875434,0.305556,0.000000,0.885714,1.481869,0.142857,0.000000,0.000000,1.0
25%,14.000000,9.000000,10.000000,3.000000,0.421325,0.887574,0.421325,0.769302,1.320000,0.250000,...,0.769231,9.912027,0.566667,0.029215,1.320000,3.152952,0.567568,0.071429,0.000000,1.0
50%,18.000000,12.000000,12.000000,4.000000,0.655555,0.901308,0.655555,0.919725,1.444444,0.476190,...,0.823529,12.802302,0.687500,0.036666,1.444444,3.340923,0.615385,0.133333,0.000000,1.0
75%,25.000000,15.000000,14.000000,6.000000,0.942809,0.914062,0.942809,1.094141,1.571429,0.608696,...,0.880000,16.398649,0.812500,0.042855,1.571429,3.540568,0.666667,0.212121,0.117647,1.0
max,40.000000,27.000000,26.000000,16.000000,2.661453,0.956633,2.661453,2.302397,2.000000,4.000000,...,0.925000,34.614674,1.000000,0.177778,2.000000,4.527247,0.913043,0.857143,0.625000,1.0


In [ ]:
import pandas as pd

# Load dataset (replace with your file path)
file_path = "dga_18_version2.csv"


# Read dataset
df = pd.read_csv(file_path)

# Count number of rows
row_count = len(df)

print(f"Number of rows in the dataset: {row_count}")


Number of rows in the dataset: 694173


In [ ]:
import pandas as pd

def filter_outliers(df, feature_cols, method="iqr", z_thresh=3, save_path=None):
    """
    Filters datapoints outside the lower/upper bound for each feature
    and optionally saves the datapoints that are within the bounds.

    Parameters:
    -----------
    df : pd.DataFrame
        Input dataset with features.
    feature_cols : list
        List of feature columns to check.
    method : str
        "iqr" (default) -> Interquartile Range method
        "zscore" -> Standard deviation based
    z_thresh : int
        Threshold for zscore method
    save_path : str or None
        If provided, saves the filtered dataset to this CSV path.

    Returns:
    --------
    pd.DataFrame : Filtered dataset (within bounds)
    pd.DataFrame : Bounds for each feature
    """
    bounds = {}
    df_filtered = df.copy()

    for col in feature_cols:
        if col not in df.columns:
            continue  # skip missing features

        series = df[col].dropna()

        if method == "iqr":
            Q1 = series.quantile(0.25)
            Q3 = series.quantile(0.75)
            IQR = Q3 - Q1
            lower = Q1 - 1.5 * IQR
            upper = Q3 + 1.5 * IQR

        elif method == "zscore":
            mean = series.mean()
            std = series.std()
            lower = mean - z_thresh * std
            upper = mean + z_thresh * std

        else:
            raise ValueError("Method must be 'iqr' or 'zscore'")

        bounds[col] = (lower, upper)

        # keep only rows within bounds
        df_filtered = df_filtered[(df_filtered[col] >= lower) & (df_filtered[col] <= upper)]

    bounds_df = pd.DataFrame(bounds, index=["Lower_Bound", "Upper_Bound"]).T

    # Save filtered dataset if save_path provided
    if save_path:
        df_filtered.to_csv(save_path, index=False)
        print(f"✅ Filtered dataset (within bounds) saved to {save_path}")

    return df_filtered.reset_index(drop=True), bounds_df


# ==== Example Usage ====
all_features = [

    #"Length","Consonant_Count","Unique_Chars","Max_Cons_Cluster","Dist_STD","Char_Gini","Char_Freq_Deviation","KL_divergence",#"Compression_ratio",
    #"Pronounceability",#"Bigram_Score",
    #"Trigram_Score","Markov_Chain_Likelihood","Bigram_Likelihood","Ngram_LM_Perplexity","Unique_Char_Ratio","Norm_Char_Freq_Var"


    #"Kolmogorov_Complexity"
    #"Max_Repeated_Char_Run"


    "Length","Consonant_Count","Unique_Chars","Max_Cons_Cluster","Dist_STD","Char_Gini","Char_Freq_Deviation","KL_divergence","Compression_ratio","Pronounceability","Bigram_Score","Trigram_Score","Markov_Chain_Likelihood","Bigram_Likelihood","Ngram_LM_Perplexity","Unique_Char_Ratio","Norm_Char_Freq_Var","Kolmogorov_Complexity","Renyi_Entropy","Min_Levenshtein_to_Popular","Keyboard_Distance_Score","Sliding_Word_Ratio"


    #"Renyi_Entropy"
    #"Min_Levenshtein_to_Popular"
    #"Keyboard_Distance_Score"

    #  "Length","Consonant_Count","Unique_Chars","Max_Cons_Cluster","Dist_STD","Char_Gini","Char_Freq_Deviation","KL_divergence","Pronounceability","Trigram_Score","Markov_Chain_Likelihood","Bigram_Likelihood","Ngram_LM_Perplexity","Unique_Char_Ratio","Norm_Char_Freq_Var"
    #"Length","Consonant_Count","Max_Cons_Cluster","Dist_STD","Char_Gini","Char_Freq_Deviation","KL_divergence","Compression_ratio","Bigram_Score","Trigram_Score","Markov_Chain_Likelihood","Ngram_LM_Perplexity","Unique_Char_Ratio","Norm_Char_Freq_Var"
    #"Char_Freq_Deviation","Unique_Char_Ratio"
    #"Length","Unique_Chars","Dist_STD","Char_Freq_Deviation","Compression_ratio","Bigram_Score","Trigram_Score","Markov_Chain_Likelihood","Unique_Char_Ratio"
    #"Compression_ratio"

   # "Length","Consonant_Count","Unique_Chars","Max_Cons_Cluster","Dist_STD","Char_Gini","Char_Freq_Deviation","KL_divergence","Compression_ratio","Pronounceability","Bigram_Score","Trigram_Score","Markov_Chain_Likelihood","Ngram_LM_Perplexity","Unique_Char_Ratio","Norm_Char_Freq_Var"
   # "Dist_STD","Markov_Chain_Likelihood"
    #"Length","Unique_Chars","Dist_STD","Char_Gini","Char_Freq_Deviation","KL_divergence","Compression_ratio","Pronounceability","Bigram_Score","Trigram_Score","Markov_Chain_Likelihood","Ngram_LM_Perplexity","Unique_Char_Ratio","Norm_Char_Freq_Var"


]

df = pd.read_csv("dga_18_version2.csv")
# Remove duplicate columns (keep first occurrence)
df = df.loc[:, ~df.columns.duplicated()]

# Save filtered data to CSV
df_filtered, bounds = filter_outliers(
    df, all_features, method="iqr", save_path="dga_18_version3.csv"
)

print(bounds)


✅ Filtered dataset (within bounds) saved to dga_18_version3.csv
                            Lower_Bound  Upper_Bound
Length                        -2.500000    41.500000
Consonant_Count                0.000000    24.000000
Unique_Chars                   4.000000    20.000000
Max_Cons_Cluster              -1.500000    10.500000
Dist_STD                      -0.360901     1.725035
Char_Gini                      0.847841     0.953795
Char_Freq_Deviation           -0.360901     1.725035
KL_divergence                  0.282044     1.581399
Compression_ratio              0.942857     1.948571
Pronounceability              -0.288043     1.146739
Bigram_Score                  -5.500000    38.500000
Trigram_Score                 -6.500000    37.500000
Markov_Chain_Likelihood     -121.822026    19.742540
Bigram_Likelihood              0.603077     1.046154
Ngram_LM_Perplexity            0.182094    26.128583
Unique_Char_Ratio              0.197917     1.181250
Norm_Char_Freq_Var             0.00

In [ ]:
from google.colab import files
df=pd.read_csv("dga_18_version3.csv")
df.describe()

,Length,Consonant_Count,Unique_Chars,Max_Cons_Cluster,Dist_STD,Char_Gini,Char_Freq_Deviation,KL_divergence,Compression_ratio,Pronounceability,...,Bigram_Likelihood,Ngram_LM_Perplexity,Unique_Char_Ratio,Norm_Char_Freq_Var,Kolmogorov_Complexity,Renyi_Entropy,Min_Levenshtein_to_Popular,Keyboard_Distance_Score,Sliding_Word_Ratio,label
count,572233.000000,572233.000000,572233.000000,572233.000000,572233.000000,572233.000000,572233.000000,572233.000000,572233.000000,572233.000000,...,572233.000000,572233.000000,572233.000000,572233.000000,572233.000000,572233.000000,572233.000000,572233.000000,572233.000000,572233.0
mean,19.848646,12.490755,12.752150,4.330890,0.725677,0.900989,0.725677,0.915040,1.448823,0.476259,...,0.827141,14.232752,0.682994,0.036260,1.448823,3.359791,0.614898,0.139552,0.059090,1.0
std,6.775358,4.141531,2.508406,2.135666,0.317764,0.018044,0.317764,0.202885,0.167894,0.213786,...,0.061562,4.634020,0.139425,0.009227,0.167894,0.259978,0.077071,0.087475,0.081459,0.0
min,9.000000,4.000000,8.000000,1.000000,0.229061,0.850000,0.229061,0.386876,0.947368,0.000000,...,0.607143,3.518005,0.305556,0.012056,0.947368,2.736966,0.419355,0.000000,0.000000,1.0
25%,16.000000,10.000000,11.000000,3.000000,0.433013,0.890000,0.433013,0.752807,1.307692,0.307692,...,0.800000,10.823575,0.560000,0.029463,1.307692,3.184425,0.567568,0.071429,0.000000,1.0
50%,19.000000,12.000000,13.000000,4.000000,0.655555,0.903226,0.655555,0.897360,1.421053,0.500000,...,0.842105,13.671606,0.684211,0.036289,1.421053,3.369234,0.615385,0.133333,0.000000,1.0
75%,26.000000,15.000000,15.000000,6.000000,0.942809,0.914127,0.942809,1.070826,1.500000,0.611111,...,0.884615,16.398649,0.812500,0.041667,1.500000,3.541659,0.666667,0.200000,0.125000,1.0
max,39.000000,24.000000,20.000000,10.000000,1.724879,0.942222,1.724879,1.396110,1.888889,1.142857,...,0.923077,25.931640,0.947368,0.063311,1.888889,4.113341,0.812500,0.423077,0.294118,1.0


In [ ]:
# dga_feature_extractor.py
import pandas as pd
import numpy as np
import re
import math
import zlib
import gzip
import string
from collections import Counter
from nltk.corpus import words
from nltk import download
from difflib import SequenceMatcher
from scipy.stats import entropy, kurtosis
from sklearn.preprocessing import normalize

# Download dictionary
download('words')
english_words = set(w.lower() for w in words.words())

VOWELS = set("aeiou")
CONSONANTS = set(string.ascii_lowercase) - VOWELS
POPULAR_TLDS = {".com", ".org", ".net", ".info", ".edu", ".gov"}
BAD_TLDS = {".xyz", ".top", ".club", ".work", ".click"}

# Example benign distribution (uniform)
BENIGN_DIST = np.ones(26) / 26

def shannon_entropy(s):
    p, _ = np.histogram(list(s), bins=range(257), density=True)
    p = p[p > 0]
    return -np.sum(p * np.log2(p))

def normalized_entropy(s):
    if not s: return 0
    return shannon_entropy(s) / math.log2(len(set(s)) + 1)

def compress_ratio(s):
    if not s: return 0
    return len(zlib.compress(s.encode())) / max(1, len(s))

def dict_word_features(domain):
    matches = [w for w in english_words if w in domain]
    longest = max((len(w) for w in matches), default=0)
    return len(matches), longest, longest / max(1,len(domain))

def sliding_dict_ratio(domain, window=4):
    n = len(domain)
    if n < window: return 0
    matches = 0
    for i in range(n - window + 1):
        if domain[i:i+window] in english_words:
            matches += 1
    return matches / (n - window + 1)

def vowel_consonant_alt(domain):
    domain = re.sub(r'[^a-z]', '', domain.lower())
    count = 0
    for i in range(1, len(domain)):
        if (domain[i] in VOWELS and domain[i-1] in CONSONANTS) or (domain[i] in CONSONANTS and domain[i-1] in VOWELS):
            count += 1
    return count

def char_gini(s):
    counts = Counter(s)
    N = sum(counts.values())
    if N == 0: return 0
    gini = 1 - sum((c/N)**2 for c in counts.values())
    return gini

def kl_divergence(s):
    counts = Counter([c for c in s if c.isalpha()])
    N = sum(counts.values())
    if N == 0: return 0
    dist = np.array([counts.get(chr(97+i),0)/N for i in range(26)])
    return entropy(dist, BENIGN_DIST)

def markov_chain_likelihood(domain):
    if len(domain) < 2: return 0
    transitions = {}
    for i in range(len(domain)-1):
        pair = (domain[i], domain[i+1])
        transitions[pair] = transitions.get(pair, 0) + 1
    total = sum(transitions.values())
    return sum(math.log((c/total)+1e-6) for c in transitions.values())

def autocorrelation_score(domain):
    if len(domain) < 2: return 0
    values = [ord(c) for c in domain]
    mean = np.mean(values)
    var = np.var(values)
    corr = sum((values[i]-mean)*(values[i+1]-mean) for i in range(len(values)-1)) / (var*(len(values)-1)+1e-6)
    return corr

def hyphen_word_match_ratio(domain):
    """Compute ratio of hyphen-separated parts that are valid dictionary words."""
    parts = domain.split("-")
    if not parts:
        return 0.0
    dict_matches = sum(1 for p in parts if p in english_words)
    return dict_matches / len(parts)

# === Extra Feature Functions ===
def renyi_entropy(s, alpha=2):
    """Rényi entropy of order alpha (default α=2)."""
    if not s:
        return 0
    counts = Counter(s)
    probs = np.array(list(counts.values())) / len(s)
    if alpha == 1:
        return -np.sum(probs * np.log2(probs))  # Shannon
    return 1 / (1 - alpha) * np.log2(np.sum(probs ** alpha))

# Load top domains (free list like Tranco or Cisco Umbrella top domains)
try:
    with open("top1k_domains.txt") as f:
        popular_domains = [line.strip().lower() for line in f.readlines()]
except FileNotFoundError:
    popular_domains = ["google.com","facebook.com","youtube.com","amazon.com","wikipedia.org"]  # fallback demo list

def min_levenshtein_to_popular(domain, topN=500):
    """Compute minimum normalized Levenshtein distance to topN popular domains."""
    domain = domain.lower()
    min_dist = 1.0
    for pd in popular_domains[:topN]:
        ratio = SequenceMatcher(None, domain, pd).ratio()
        dist = 1 - ratio  # 0 = identical, 1 = very different
        if dist < min_dist:
            min_dist = dist
    return min_dist

# Keyboard adjacency mapping
KEYBOARD_NEIGHBORS = {
    'q': "was", 'w': "qase", 'e': "wsdr", 'r': "edft", 't': "rfgy",
    'y': "tghu", 'u': "yhj", 'i': "ujk", 'o': "ikl", 'p': "ol",
    'a': "qwsz", 's': "qwedxza", 'd': "erfcxs", 'f': "rtgvcd", 'g': "tyhbvf",
    'h': "yujnbg", 'j': "uikmnh", 'k': "iolmj", 'l': "opk",
    'z': "asx", 'x': "zsdc", 'c': "xdfv", 'v': "cfgb", 'b': "vghn",
    'n': "bhjm", 'm': "njk"
}

def keyboard_distance_score(domain):
    """Average keyboard adjacency match score between consecutive characters."""
    domain = re.sub(r'[^a-z]', '', domain.lower())
    if len(domain) < 2:
        return 0
    score = 0
    for i in range(len(domain)-1):
        if domain[i+1] in KEYBOARD_NEIGHBORS.get(domain[i], ""):
            score += 1
    return score / (len(domain)-1)

# === Updated extract_features ===
def extract_features(domain):
    if not isinstance(domain, str):
        domain = str(domain)
    domain = domain.lower()
    name, _, tld = domain.rpartition(".")
    if not name:
        name = domain







#"Length","Consonant_Count","Unique_Chars","Max_Cons_Cluster","Dist_STD","Char_Gini","Char_Freq_Deviation","KL_divergence","Compression_ratio","Pronounceability","Bigram_Score","Trigram_Score","Markov_Chain_Likelihood","Bigram_Likelihood","Ngram_LM_Perplexity","Unique_Char_Ratio","Norm_Char_Freq_Var"

    feats = {}

     # === Advanced Features Added ===
    #feats["Renyi_Entropy"] = renyi_entropy(domain, alpha=2)
    #feats["Min_Levenshtein_to_Popular"] = min_levenshtein_to_popular(domain)
    feats["Keyboard_Distance_Score"] = keyboard_distance_score(domain)


    #feats["Length"] = len(domain)
    #feats["Consonant_Count"] = sum(c in CONSONANTS for c in domain)
    #feats["Unique_Chars"] = len(set(domain))
    #feats["Max_Cons_Cluster"] = max((len(m.group()) for m in re.finditer(r'[bcdfghjklmnpqrstvwxyz]+', domain)), default=0)

    # Info-theoretic
    #feats["Dist_STD"] = np.std([domain.count(c) for c in set(domain)])
    #feats["Char_Gini"] = char_gini(domain)
    #feats["Char_Freq_Deviation"] = np.std(list(Counter(domain).values()))
    #feats["KL_divergence"] = kl_divergence(domain)
    #feats["Compression_ratio"] = compress_ratio(domain)

    # Pronounceability
    #feats["Pronounceability"] = sum(c in VOWELS for c in domain) / (sum(c in CONSONANTS for c in domain) + 1)

    # N-gram / LM
    #feats["Bigram_Score"] = sum(1 for i in range(len(domain) - 1) if domain[i].isalpha() and domain[i + 1].isalpha())
    #feats["Trigram_Score"] = sum(1 for i in range(len(domain) - 2) if domain[i].isalpha() and domain[i + 2].isalpha())
    #feats["Markov_Chain_Likelihood"] = markov_chain_likelihood(domain)
    #feats["Bigram_Likelihood"] = sum(1 for i in range(len(domain)-1) if domain[i].isalpha() and domain[i+1].isalpha())/max(1,len(domain))
    #feats["Ngram_LM_Perplexity"] = math.exp(-markov_chain_likelihood(domain) / max(1, len(domain)))

    # Structural/pattern
    #feats["Unique_Char_Ratio"] = len(set(domain)) / max(1, len(domain))
    #feats["Norm_Char_Freq_Var"] = np.std(list(Counter(domain).values())) / max(1, len(domain))



  #feats["Kolmogorov_Complexity"] = compress_ratio(domain)

    #feats["Max_Repeated_Char_Run"] = max( (len(m.group()) for m in re.finditer(r'(.)\1+', domain)), default=0)





    ##feats["Digit_Count"] = sum(c.isdigit() for c in domain)
    ##feats["Digit_Ratio"] = feats["Digit_Count"]/max(1,len(domain))
    ##feats["Max_Consec_Digits"] = max((len(m.group()) for m in re.finditer(r'\d+', domain)), default=0)
    ##feats["Vowel_Count"] = sum(c in VOWELS for c in domain)
    ##feats["Has_Hyphen"] = "-" in domain
    ##feats["Hyphen_Placement_Score"] = 1 if domain.startswith("-") or domain.endswith("-") else 0
    ##feats["Hyphen_Word_Match_Ratio"] = hyphen_word_match_ratio(domain)
    ##feats["Subdomain_Depth"] = domain.count(".")
    ##feats["Entropy"] = shannon_entropy(domain)
    #feats["Normalized_Entropy"] = normalized_entropy(domain)
    ##feats["Vowel_Entropy"] = shannon_entropy("".join([c for c in domain if c in VOWELS]))
    ##feats["Consonant_Entropy"] = shannon_entropy("".join([c for c in domain if c not in VOWELS and c.isalpha()]))
    #dwc, lwl, lwr = dict_word_features(domain)
    #feats["Dict_Word_Count"] = dwc
    #feats["Longest_Word_Len"] = lwl
    #feats["Longest_Word_Ratio"] = lwr
    #feats["Word_Match_Ratio"] = dwc/max(1,len(domain.split(".")))
    #feats["Sliding_Word_Ratio"] = sliding_dict_ratio(domain)
    ##feats["Vowel_Consonant_Alt"] = vowel_consonant_alt(domain)
    #feats["Lexical_Complexity"] = shannon_entropy(domain)*len(set(domain))/max(1,len(domain))
    #feats["Bigram_Likelihood"] = sum(1 for i in range(len(domain)-1) if domain[i].isalpha() and domain[i+1].isalpha())/max(1,len(domain))
    #feats["TLD_Common"] = 1 if f".{tld}" in POPULAR_TLDS else 0
    ##feats["TLD_Bad_Score"] = 1 if f".{tld}" in BAD_TLDS else 0
    ##feats["Popular_Domain"] = 0  # needs Alexa dataset
    ##feats["Alexa_popularity_bucket"] = -1
    #feats["Repeat_Chars"] = max((len(m.group()) for m in re.finditer(r'(.)\1+', domain)), default=0)
    ##feats["Max_Repeated_Char_Run"] = feats["Repeat_Chars"]
    ##feats["Digit_Letter_Alt_Count"] = sum((domain[i].isdigit() != domain[i+1].isdigit()) for i in range(len(domain)-1))
    #feats["Palindromic_Substring_Ratio"] = sum(domain[i:j]==domain[i:j][::-1] for i in range(len(domain)) for j in range(i+2,len(domain)+1))/max(1,len(domain))
    ##feats["First_Last_Char_Type"] = int(domain[0].isalpha()) + int(domain[-1].isalpha())
    ##feats["Shannon_Window_Entropy"] = np.mean([shannon_entropy(domain[i:i+4]) for i in range(len(domain)-3)]) if len(domain)>=4 else  shannon_entropy(domain)
    ##feats["Markov_Cross_Entropy"] = shannon_entropy(domain) -  markov_chain_likelihood(domain)
    ##feats["Permutation_Entropy"] = shannon_entropy(domain)/max(1,math.log2(len(domain)))
    #feats["RunLength_Entropy"] = shannon_entropy("".join([str(len(m.group())) for m in re.finditer(r'(.)\1*', domain)]))
    #feats["Dict_Word_Coverage"] = dwc/max(1,len(domain))
    #feats["Longest_Word_Suffix"] = max((len(w) for w in english_words if domain.endswith(w)), default=0)
    #feats["Word_Boundary_Count"] = domain.count("-")
    #feats["Syllable_Count"] = sum(domain.count(v) for v in VOWELS)
    #feats["TLD_Length"] = len(tld)
    #feats["TLD_Rarity_Score"] = 0 if f".{tld}" in POPULAR_TLDS else 1
    #feats["Autocorrelation_Score"] = autocorrelation_score(domain)
    #feats["Hurst_Exponent"] = 0.5  # placeholder

    return feats

# === Run on a dataset ===
df = pd.read_csv("ndga_version2.csv")  # must have "domain" column
features = df["domain"].apply(extract_features)
feat_df = pd.DataFrame(list(features))
out = pd.concat([df, feat_df], axis=1)
out.to_csv("ndga_18_version1.csv", index=False)
print("✅ Features saved to domains_with_features.csv")

[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Package words is already up-to-date!


✅ Features saved to domains_with_features.csv


In [ ]:
# dga_feature_extractor.py
import pandas as pd
import numpy as np
import re
import math
import zlib
import gzip
import string
from collections import Counter
from nltk.corpus import words
from nltk import download
from difflib import SequenceMatcher
from scipy.stats import entropy, kurtosis
from sklearn.preprocessing import normalize

# Download dictionary
download('words')
english_words = set(w.lower() for w in words.words())

VOWELS = set("aeiou")
CONSONANTS = set(string.ascii_lowercase) - VOWELS
POPULAR_TLDS = {".com", ".org", ".net", ".info", ".edu", ".gov"}
BAD_TLDS = {".xyz", ".top", ".club", ".work", ".click"}

# Example benign distribution (uniform)
BENIGN_DIST = np.ones(26) / 26

def shannon_entropy(s):
    p, _ = np.histogram(list(s), bins=range(257), density=True)
    p = p[p > 0]
    return -np.sum(p * np.log2(p))

def normalized_entropy(s):
    if not s: return 0
    return shannon_entropy(s) / math.log2(len(set(s)) + 1)

def compress_ratio(s):
    if not s: return 0
    return len(zlib.compress(s.encode())) / max(1, len(s))

def dict_word_features(domain):
    matches = [w for w in english_words if w in domain]
    longest = max((len(w) for w in matches), default=0)
    return len(matches), longest, longest / max(1,len(domain))

def sliding_dict_ratio(domain, window=4):
    n = len(domain)
    if n < window: return 0
    matches = 0
    for i in range(n - window + 1):
        if domain[i:i+window] in english_words:
            matches += 1
    return matches / (n - window + 1)

def vowel_consonant_alt(domain):
    domain = re.sub(r'[^a-z]', '', domain.lower())
    count = 0
    for i in range(1, len(domain)):
        if (domain[i] in VOWELS and domain[i-1] in CONSONANTS) or (domain[i] in CONSONANTS and domain[i-1] in VOWELS):
            count += 1
    return count

def char_gini(s):
    counts = Counter(s)
    N = sum(counts.values())
    if N == 0: return 0
    gini = 1 - sum((c/N)**2 for c in counts.values())
    return gini

def kl_divergence(s):
    counts = Counter([c for c in s if c.isalpha()])
    N = sum(counts.values())
    if N == 0: return 0
    dist = np.array([counts.get(chr(97+i),0)/N for i in range(26)])
    return entropy(dist, BENIGN_DIST)

def markov_chain_likelihood(domain):
    if len(domain) < 2: return 0
    transitions = {}
    for i in range(len(domain)-1):
        pair = (domain[i], domain[i+1])
        transitions[pair] = transitions.get(pair, 0) + 1
    total = sum(transitions.values())
    return sum(math.log((c/total)+1e-6) for c in transitions.values())

def autocorrelation_score(domain):
    if len(domain) < 2: return 0
    values = [ord(c) for c in domain]
    mean = np.mean(values)
    var = np.var(values)
    corr = sum((values[i]-mean)*(values[i+1]-mean) for i in range(len(values)-1)) / (var*(len(values)-1)+1e-6)
    return corr

def hyphen_word_match_ratio(domain):
    """Compute ratio of hyphen-separated parts that are valid dictionary words."""
    parts = domain.split("-")
    if not parts:
        return 0.0
    dict_matches = sum(1 for p in parts if p in english_words)
    return dict_matches / len(parts)

# === Extra Feature Functions ===
def renyi_entropy(s, alpha=2):
    """Rényi entropy of order alpha (default α=2)."""
    if not s:
        return 0
    counts = Counter(s)
    probs = np.array(list(counts.values())) / len(s)
    if alpha == 1:
        return -np.sum(probs * np.log2(probs))  # Shannon
    return 1 / (1 - alpha) * np.log2(np.sum(probs ** alpha))

# Load top domains (free list like Tranco or Cisco Umbrella top domains)
try:
    with open("top1k_domains.txt") as f:
        popular_domains = [line.strip().lower() for line in f.readlines()]
except FileNotFoundError:
    popular_domains = ["google.com","facebook.com","youtube.com","amazon.com","wikipedia.org"]  # fallback demo list

def min_levenshtein_to_popular(domain, topN=500):
    """Compute minimum normalized Levenshtein distance to topN popular domains."""
    domain = domain.lower()
    min_dist = 1.0
    for pd in popular_domains[:topN]:
        ratio = SequenceMatcher(None, domain, pd).ratio()
        dist = 1 - ratio  # 0 = identical, 1 = very different
        if dist < min_dist:
            min_dist = dist
    return min_dist

# Keyboard adjacency mapping
KEYBOARD_NEIGHBORS = {
    'q': "was", 'w': "qase", 'e': "wsdr", 'r': "edft", 't': "rfgy",
    'y': "tghu", 'u': "yhj", 'i': "ujk", 'o': "ikl", 'p': "ol",
    'a': "qwsz", 's': "qwedxza", 'd': "erfcxs", 'f': "rtgvcd", 'g': "tyhbvf",
    'h': "yujnbg", 'j': "uikmnh", 'k': "iolmj", 'l': "opk",
    'z': "asx", 'x': "zsdc", 'c': "xdfv", 'v': "cfgb", 'b': "vghn",
    'n': "bhjm", 'm': "njk"
}

def keyboard_distance_score(domain):
    """Average keyboard adjacency match score between consecutive characters."""
    domain = re.sub(r'[^a-z]', '', domain.lower())
    if len(domain) < 2:
        return 0
    score = 0
    for i in range(len(domain)-1):
        if domain[i+1] in KEYBOARD_NEIGHBORS.get(domain[i], ""):
            score += 1
    return score / (len(domain)-1)

# === Updated extract_features ===
def extract_features(domain):
    if not isinstance(domain, str):
        domain = str(domain)
    domain = domain.lower()
    name, _, tld = domain.rpartition(".")
    if not name:
        name = domain






#"Length","Consonant_Count","Unique_Chars","Max_Cons_Cluster","Dist_STD","Char_Gini","Char_Freq_Deviation","KL_divergence","Compression_ratio","Pronounceability","Bigram_Score","Trigram_Score","Markov_Chain_Likelihood","Bigram_Likelihood","Ngram_LM_Perplexity","Unique_Char_Ratio","Norm_Char_Freq_Var","Kolmogorov_Complexity","Renyi_Entropy","Min_Levenshtein_to_Popular","Keyboard_Distance_Score","Sliding_Word_Ratio"

    feats = {}



    feats["Length"] = len(domain)
    feats["Consonant_Count"] = sum(c in CONSONANTS for c in domain)
    feats["Unique_Chars"] = len(set(domain))
    feats["Max_Cons_Cluster"] = max((len(m.group()) for m in re.finditer(r'[bcdfghjklmnpqrstvwxyz]+', domain)), default=0)

    # Info-theoretic
    feats["Dist_STD"] = np.std([domain.count(c) for c in set(domain)])
    feats["Char_Gini"] = char_gini(domain)
    feats["Char_Freq_Deviation"] = np.std(list(Counter(domain).values()))
    feats["KL_divergence"] = kl_divergence(domain)
    feats["Compression_ratio"] = compress_ratio(domain)

    # Pronounceability
    feats["Pronounceability"] = sum(c in VOWELS for c in domain) / (sum(c in CONSONANTS for c in domain) + 1)

    # N-gram / LM
    feats["Bigram_Score"] = sum(1 for i in range(len(domain) - 1) if domain[i].isalpha() and domain[i + 1].isalpha())
    feats["Trigram_Score"] = sum(1 for i in range(len(domain) - 2) if domain[i].isalpha() and domain[i + 2].isalpha())
    feats["Markov_Chain_Likelihood"] = markov_chain_likelihood(domain)
    feats["Bigram_Likelihood"] = sum(1 for i in range(len(domain)-1) if domain[i].isalpha() and domain[i+1].isalpha())/max(1,len(domain))
    feats["Ngram_LM_Perplexity"] = math.exp(-markov_chain_likelihood(domain) / max(1, len(domain)))

    # Structural/pattern
    feats["Unique_Char_Ratio"] = len(set(domain)) / max(1, len(domain))
    feats["Norm_Char_Freq_Var"] = np.std(list(Counter(domain).values())) / max(1, len(domain))



    feats["Kolmogorov_Complexity"] = compress_ratio(domain)

   # === Advanced Features Added ===
    feats["Renyi_Entropy"] = renyi_entropy(domain, alpha=2)
    feats["Min_Levenshtein_to_Popular"] = min_levenshtein_to_popular(domain)
    feats["Keyboard_Distance_Score"] = keyboard_distance_score(domain)
    feats["Sliding_Word_Ratio"] = sliding_dict_ratio(domain)

    #feats["Max_Repeated_Char_Run"] = max( (len(m.group()) for m in re.finditer(r'(.)\1+', domain)), default=0)





    ##feats["Digit_Count"] = sum(c.isdigit() for c in domain)
    ##feats["Digit_Ratio"] = feats["Digit_Count"]/max(1,len(domain))
    ##feats["Max_Consec_Digits"] = max((len(m.group()) for m in re.finditer(r'\d+', domain)), default=0)
    ##feats["Vowel_Count"] = sum(c in VOWELS for c in domain)
    ##feats["Has_Hyphen"] = "-" in domain
    ##feats["Hyphen_Placement_Score"] = 1 if domain.startswith("-") or domain.endswith("-") else 0
    ##feats["Hyphen_Word_Match_Ratio"] = hyphen_word_match_ratio(domain)
    ##feats["Subdomain_Depth"] = domain.count(".")
    ##feats["Entropy"] = shannon_entropy(domain)
    #feats["Normalized_Entropy"] = normalized_entropy(domain)
    ##feats["Vowel_Entropy"] = shannon_entropy("".join([c for c in domain if c in VOWELS]))
    ##feats["Consonant_Entropy"] = shannon_entropy("".join([c for c in domain if c not in VOWELS and c.isalpha()]))
    #dwc, lwl, lwr = dict_word_features(domain)
    #feats["Dict_Word_Count"] = dwc
    #feats["Longest_Word_Len"] = lwl
    #feats["Longest_Word_Ratio"] = lwr
    #feats["Word_Match_Ratio"] = dwc/max(1,len(domain.split(".")))
    #feats["Sliding_Word_Ratio"] = sliding_dict_ratio(domain)
    ##feats["Vowel_Consonant_Alt"] = vowel_consonant_alt(domain)
    #feats["Lexical_Complexity"] = shannon_entropy(domain)*len(set(domain))/max(1,len(domain))
    #feats["Bigram_Likelihood"] = sum(1 for i in range(len(domain)-1) if domain[i].isalpha() and domain[i+1].isalpha())/max(1,len(domain))
    #feats["TLD_Common"] = 1 if f".{tld}" in POPULAR_TLDS else 0
    ##feats["TLD_Bad_Score"] = 1 if f".{tld}" in BAD_TLDS else 0
    ##feats["Popular_Domain"] = 0  # needs Alexa dataset
    ##feats["Alexa_popularity_bucket"] = -1
    #feats["Repeat_Chars"] = max((len(m.group()) for m in re.finditer(r'(.)\1+', domain)), default=0)
    ##feats["Max_Repeated_Char_Run"] = feats["Repeat_Chars"]
    ##feats["Digit_Letter_Alt_Count"] = sum((domain[i].isdigit() != domain[i+1].isdigit()) for i in range(len(domain)-1))
    #feats["Palindromic_Substring_Ratio"] = sum(domain[i:j]==domain[i:j][::-1] for i in range(len(domain)) for j in range(i+2,len(domain)+1))/max(1,len(domain))
    ##feats["First_Last_Char_Type"] = int(domain[0].isalpha()) + int(domain[-1].isalpha())
    ##feats["Shannon_Window_Entropy"] = np.mean([shannon_entropy(domain[i:i+4]) for i in range(len(domain)-3)]) if len(domain)>=4 else  shannon_entropy(domain)
    ##feats["Markov_Cross_Entropy"] = shannon_entropy(domain) -  markov_chain_likelihood(domain)
    ##feats["Permutation_Entropy"] = shannon_entropy(domain)/max(1,math.log2(len(domain)))
    #feats["RunLength_Entropy"] = shannon_entropy("".join([str(len(m.group())) for m in re.finditer(r'(.)\1*', domain)]))
    #feats["Dict_Word_Coverage"] = dwc/max(1,len(domain))
    #feats["Longest_Word_Suffix"] = max((len(w) for w in english_words if domain.endswith(w)), default=0)
    #feats["Word_Boundary_Count"] = domain.count("-")
    #feats["Syllable_Count"] = sum(domain.count(v) for v in VOWELS)
    #feats["TLD_Length"] = len(tld)
    #feats["TLD_Rarity_Score"] = 0 if f".{tld}" in POPULAR_TLDS else 1
    #feats["Autocorrelation_Score"] = autocorrelation_score(domain)
    #feats["Hurst_Exponent"] = 0.5  # placeholder

    return feats

# === Run on a dataset ===
df = pd.read_csv("ndga_version2.csv")  # must have "domain" column
features = df["domain"].apply(extract_features)
feat_df = pd.DataFrame(list(features))
out = pd.concat([df, feat_df], axis=1)
out.to_csv("ndga_18_version1.csv", index=False)
print("✅ Features saved to domains_with_features.csv")

[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Package words is already up-to-date!


✅ Features saved to domains_with_features.csv


In [ ]:

from google.colab import files

df=pd.read_csv("ndga_18_version1.csv")

df.describe()

,label,Keyboard_Distance_Score
count,1000018.0,1.000018e+06
mean,0.0,1.289533e-01
std,0.0,1.136739e-01
min,0.0,0.000000e+00
25%,0.0,0.000000e+00
50%,0.0,1.111111e-01
75%,0.0,2.000000e-01
max,0.0,1.000000e+00


In [ ]:
import pandas as pd

def reorder_columns(input_csv, output_csv, desired_order):
    """
    Reorder the columns of a CSV file based on user-defined order.

    Parameters:
    - input_csv (str): Path to input CSV file.
    - output_csv (str): Path to output CSV file with reordered columns.
    - desired_order (list): List of column names in the desired order.
    """
    # Load dataset
    df = pd.read_csv(input_csv)

    # Check which desired columns exist
    available_columns = [col for col in desired_order if col in df.columns]

    # Add missing columns (if any were not in df)
    missing_columns = [col for col in desired_order if col not in df.columns]
    for col in missing_columns:
        df[col] = None  # Fill with None or default values

    # Reorder
    df = df[available_columns + [col for col in df.columns if col not in available_columns]]

    # Save output
    df.to_csv(output_csv, index=False)
    print(f"✅ Reordered CSV saved as {output_csv}")


# =====================
# Example usage
# =====================

# Suppose your dataset has 58 features + "domain" + "label"
input_csv = "ndga_18_version1.csv"
output_csv = "ndga_18_version2.csv"

# User-defined column order (just an example)
desired_order = [

     # "domain","Char_Freq_Deviation","Unique_Char_Ratio","label"
      #"domain","Length","Consonant_Count","Unique_Chars","Max_Cons_Cluster","Dist_STD","Char_Gini","Char_Freq_Deviation","KL_divergence","Compression_ratio","Pronounceability","Bigram_Score","Trigram_Score","Markov_Chain_Likelihood","Bigram_Likelihood","Ngram_LM_Perplexity","Unique_Char_Ratio","Norm_Char_Freq_Var","label"
     #"domain","Length","Consonant_Count","Unique_Chars","Max_Cons_Cluster","Dist_STD","Char_Gini","Char_Freq_Deviation","KL_divergence",#"Compression_ratio",
     #"Pronounceability",#"Bigram_Score",
     #"Trigram_Score","Markov_Chain_Likelihood","Bigram_Likelihood","Ngram_LM_Perplexity","Unique_Char_Ratio","Norm_Char_Freq_Var","label"

     "domain","Keyboard_Distance_Score","label"

     #"Min_Levenshtein_to_Popular"


     #"domain","Length","Consonant_Count","Unique_Chars","Max_Cons_Cluster","Dist_STD","Char_Gini","Char_Freq_Deviation","KL_divergence","Pronounceability","Trigram_Score","Markov_Chain_Likelihood","Bigram_Likelihood","Ngram_LM_Perplexity","Unique_Char_Ratio","Norm_Char_Freq_Var","label"
     #"domain","Length","Consonant_Count","Max_Cons_Cluster","Dist_STD","Char_Gini","Char_Freq_Deviation","KL_divergence","Compression_ratio","Bigram_Score","Trigram_Score","Markov_Chain_Likelihood","Ngram_LM_Perplexity","Unique_Char_Ratio","Norm_Char_Freq_Var","label"
    #"domain","Length","Unique_Chars","Dist_STD","Char_Freq_Deviation","Compression_ratio","Bigram_Score","Trigram_Score","Markov_Chain_Likelihood","Unique_Char_Ratio","label"
    #Digit_Count", "label",
   # "domain","Length","Consonant_Count","Unique_Chars","Max_Cons_Cluster","Dist_STD","Char_Gini","Char_Freq_Deviation","KL_divergence","Compression_ratio","Pronounceability","Bigram_Score","Trigram_Score","Markov_Chain_Likelihood","Ngram_LM_Perplexity","Unique_Char_Ratio","Norm_Char_Freq_Var","label"
    #"Length", "Digit_Ratio", "Max_Consec_Digits","Vowel_Count", , , "KL-divergence from benign char dist",
    #"Compression ratio", "Pronounceability", "Bigram_Score", "Trigram_Score",
    #"Alexa popularity bucket", "Homoglyph/brand-similarity (lookalike)"
]

# Reorder dataset
reorder_columns(input_csv, output_csv, desired_order)


✅ Reordered CSV saved as ndga_18_version2.csv


In [ ]:
import pandas as pd

# Load dataset (replace with your file path)
file_path = "ndga_18_version2.csv"


# Read dataset
df = pd.read_csv(file_path)

# Count number of rows
row_count = len(df)

print(f"Number of rows in the dataset: {row_count}")


Number of rows in the dataset: 1000018


In [ ]:
from google.colab import files
df=pd.read_csv("ndga_18_version2.csv")
df.describe()

,Keyboard_Distance_Score,label
count,1.000018e+06,1000018.0
mean,1.289533e-01,0.0
std,1.136739e-01,0.0
min,0.000000e+00,0.0
25%,0.000000e+00,0.0
50%,1.111111e-01,0.0
75%,2.000000e-01,0.0
max,1.000000e+00,0.0


In [ ]:
import pandas as pd

def filter_outliers(df, feature_cols, method="iqr", z_thresh=3, save_path=None):
    """
    Filters datapoints outside the lower/upper bound for each feature
    and optionally saves the datapoints that are within the bounds.

    Parameters:
    -----------
    df : pd.DataFrame
        Input dataset with features.
    feature_cols : list
        List of feature columns to check.
    method : str
        "iqr" (default) -> Interquartile Range method
        "zscore" -> Standard deviation based
    z_thresh : int
        Threshold for zscore method
    save_path : str or None
        If provided, saves the filtered dataset to this CSV path.

    Returns:
    --------
    pd.DataFrame : Filtered dataset (within bounds)
    pd.DataFrame : Bounds for each feature
    """
    bounds = {}
    df_filtered = df.copy()

    for col in feature_cols:
        if col not in df.columns:
            continue  # skip missing features

        series = df[col].dropna()

        if method == "iqr":
            Q1 = series.quantile(0.25)
            Q3 = series.quantile(0.75)
            IQR = Q3 - Q1
            lower = Q1 - 1.5 * IQR
            upper = Q3 + 1.5 * IQR

        elif method == "zscore":
            mean = series.mean()
            std = series.std()
            lower = mean - z_thresh * std
            upper = mean + z_thresh * std

        else:
            raise ValueError("Method must be 'iqr' or 'zscore'")

        bounds[col] = (lower, upper)

        # keep only rows within bounds
        df_filtered = df_filtered[(df_filtered[col] >= lower) & (df_filtered[col] <= upper)]

    bounds_df = pd.DataFrame(bounds, index=["Lower_Bound", "Upper_Bound"]).T

    # Save filtered dataset if save_path provided
    if save_path:
        df_filtered.to_csv(save_path, index=False)
        print(f"✅ Filtered dataset (within bounds) saved to {save_path}")

    return df_filtered.reset_index(drop=True), bounds_df


# ==== Example Usage ====
all_features = [

    #"Char_Freq_Deviation","Unique_Char_Ratio"
    #"Length","Consonant_Count","Max_Cons_Cluster","Dist_STD","Char_Gini","Char_Freq_Deviation","KL_divergence","Compression_ratio","Bigram_Score","Trigram_Score","Markov_Chain_Likelihood","Ngram_LM_Perplexity","Unique_Char_Ratio","Norm_Char_Freq_Var"
    #"Dist_STD","Markov_Chain_Likelihood"

   #"Length","Consonant_Count","Unique_Chars","Max_Cons_Cluster","Dist_STD","Char_Gini","Char_Freq_Deviation","KL_divergence",#"Compression_ratio",
   # "Pronounceability",#"Bigram_Score",
    #"Trigram_Score","Markov_Chain_Likelihood","Bigram_Likelihood","Ngram_LM_Perplexity","Unique_Char_Ratio","Norm_Char_Freq_Var"
    #"Min_Levenshtein_to_Popular"
    "Keyboard_Distance_Score"
    #"Renyi_Entropy"
    #"Kolmogorov_Complexity"
    #"Max_Repeated_Char_Run"
    #"
    #

    #"Length","Consonant_Count","Unique_Chars","Max_Cons_Cluster","Dist_STD","Char_Gini","Char_Freq_Deviation","KL_divergence","Pronounceability","Trigram_Score","Markov_Chain_Likelihood","Bigram_Likelihood","Ngram_LM_Perplexity","Unique_Char_Ratio","Norm_Char_Freq_Var"
    #"Length","Unique_Chars","Dist_STD","Char_Freq_Deviation","Compression_ratio","Bigram_Score","Trigram_Score","Markov_Chain_Likelihood","Unique_Char_Ratio"
   # "Vowel_Consonant_Alt"

   # "Length","Consonant_Count","Unique_Chars","Max_Cons_Cluster","Dist_STD","Char_Gini","Char_Freq_Deviation","KL_divergence","Compression_ratio","Pronounceability","Bigram_Score",
    #"Trigram_Score","Markov_Chain_Likelihood","Ngram_LM_Perplexity","Unique_Char_Ratio","Norm_Char_Freq_Var"

]

df = pd.read_csv("ndga_18_version2.csv")

# Save filtered data to CSV
df_filtered, bounds = filter_outliers(
    df, all_features, method="iqr", save_path="ndga_18_version3.csv"
)

print(bounds)


✅ Filtered dataset (within bounds) saved to ndga_18_version3.csv
                         Lower_Bound  Upper_Bound
Keyboard_Distance_Score         -0.3          0.5


In [ ]:

import pandas as pd

def select_entries(file, num_entries, output_file="ndga_18_version4.csv"):
    """
    Select the user-specified number of entries (rows) from a dataset.

    Args:
        file (str): Path to the CSV file.

        num_entries (int): Number of rows to select.
        output_file (str): File to save the selected rows.
    """
    # Load dataset
    df = pd.read_csv(file)

    # Select first 'num_entries' rows
    selected_df = df.head(num_entries)

    # Save result
    selected_df.to_csv(output_file, index=False)


    return selected_df


# Example usage:
selected = select_entries("ndga_18_version3.csv", 680168 )   # get first 100 rows
print(selected)


                       domain  Keyboard_Distance_Score  label
0                  0-0.online                 0.000000      0
1                  0-0szd.com                 0.400000      0
2                      0-1.ir                 0.000000      0
3       0-100australia.com.au                 0.076923      0
4              0-18klinik.com                 0.250000      0
...                       ...                      ...    ...
680163            plumpos.com                 0.222222      0
680164            plumpwc.com                 0.111111      0
680165        plumpyfield.com                 0.153846      0
680166               plumr.de                 0.500000      0
680167        plums-plain.com                 0.166667      0

[680168 rows x 3 columns]


In [ ]:
from google.colab import files
df=pd.read_csv("ndga_18_version4.csv")
df.describe()

,Keyboard_Distance_Score,label
count,680168.000000,680168.0
mean,0.122901,0.0
std,0.108254,0.0
min,0.000000,0.0
25%,0.000000,0.0
50%,0.111111,0.0
75%,0.187500,0.0
max,0.500000,0.0


In [ ]:
import pandas as pd

def combine_datasets(file1, file2, output_file="combined.csv"):
    """
    Combine two datasets without altering datapoints.
    The header of the second dataset is removed automatically.
    """
    # Load first dataset normally (with header)
    df1 = pd.read_csv(file1)

    # Load second dataset as raw, then reassign columns from df1
    df2 = pd.read_csv(file2, header=None, skiprows=1)
    df2.columns = df1.columns  # assign the same header as df1

    # Concatenate without altering datapoints
    combined_df = pd.concat([df1, df2], ignore_index=True)

    # Save to CSV
    combined_df.to_csv(output_file, index=False)
    return combined_df



# Example usage:
combined = combine_datasets("dga_18_version3.csv", "ndga_18_version4.csv")
print(combined.head())


                domain  Keyboard_Distance_Score  label
0  ofdhiydrrttpblp.com                 0.235294      1
1  puciftnfkplcbhp.net                 0.176471      1
2  bowjjxxnhkyvygk.biz                 0.117647      1
3   osvwkptpwqyiqen.ru                 0.062500      1
4  cpmjpnwdgbxyyql.org                 0.176471      1


In [ ]:
!pip install xgboost
!pip install scikit-learn

In [ ]:
import pandas as pd
import numpy as np
import re
import math
import zlib
import string
import joblib
from collections import Counter
from difflib import SequenceMatcher
from xgboost import XGBClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score
from nltk.corpus import words
from nltk import download
from scipy.stats import entropy

# -------------------------
# Download English words
# -------------------------
try:
    download('words')
    ENGLISH_WORDS = set(w.lower() for w in words.words())
except:
    ENGLISH_WORDS = {"test","shop","login","home","mail","secure"}  # fallback

# -------------------------
# Constants
# -------------------------
VOWELS = set("aeiou")
CONSONANTS = set(string.ascii_lowercase) - VOWELS
POPULAR_TLDS = {".com", ".org", ".net", ".info", ".edu", ".gov"}
BAD_TLDS = {".xyz", ".top", ".club", ".work", ".click"}
BENIGN_DIST = np.ones(26)/26  # uniform for KL divergence

# -------------------------
# Load top domains
# -------------------------
try:
    with open("top1k_domains.txt") as f:
        POPULAR_DOMAINS = [line.strip().lower() for line in f.readlines()]
except FileNotFoundError:
    POPULAR_DOMAINS = ["google.com","facebook.com","youtube.com","amazon.com","wikipedia.org"]

# -------------------------
# Feature Functions
# -------------------------
def shannon_entropy(s):
    if not s: return 0
    p, _ = np.histogram(list(s), bins=range(257), density=True)
    p = p[p>0]
    return -np.sum(p*np.log2(p))

def normalized_entropy(s):
    if not s: return 0
    return shannon_entropy(s)/math.log2(len(set(s))+1)

def compress_ratio(s):
    if not s: return 0
    return len(zlib.compress(s.encode()))/max(1,len(s))

def renyi_entropy(s, alpha=2):
    if not s: return 0
    counts = Counter(s)
    probs = np.array(list(counts.values())) / len(s)
    if alpha == 1:
        return -np.sum(probs*np.log2(probs))
    return 1/(1-alpha) * np.log2(np.sum(probs**alpha))

def min_levenshtein_to_popular(domain, topN=500):
    domain = domain.lower()
    min_dist = 1.0
    for pd in POPULAR_DOMAINS[:topN]:
        ratio = SequenceMatcher(None, domain, pd).ratio()
        dist = 1 - ratio
        if dist < min_dist:
            min_dist = dist
    return min_dist

# -------------------------
# Keyboard adjacency mapping
# -------------------------
KEYBOARD_NEIGHBORS = {
    'q':"was",'w':"qase",'e':"wsdr",'r':"edft",'t':"rfgy",
    'y':"tghu",'u':"yhj",'i':"ujk",'o':"ikl",'p':"ol",
    'a':"qwsz",'s':"qwedxza",'d':"erfcxs",'f':"rtgvcd",'g':"tyhbvf",
    'h':"yujnbg",'j':"uikmnh",'k':"iolmj",'l':"opk",
    'z':"asx",'x':"zsdc",'c':"xdfv",'v':"cfgb",'b':"vghn",
    'n':"bhjm",'m':"njk"
}

def keyboard_distance_score(domain):
    domain = re.sub(r'[^a-z]','',domain.lower())
    if len(domain)<2: return 0
    score=0
    for i in range(len(domain)-1):
        if domain[i+1] in KEYBOARD_NEIGHBORS.get(domain[i],""):
            score+=1
    return score/(len(domain)-1)

# -------------------------
# Word / Lexical Features
# -------------------------
def dict_word_features(domain):
    matches = [w for w in ENGLISH_WORDS if w in domain]
    longest = max((len(w) for w in matches), default=0)
    return len(matches), longest, longest/max(1,len(domain))

def sliding_dict_ratio(domain, window=4):
    n=len(domain)
    if n<window: return 0
    matches=0
    for i in range(n-window+1):
        if domain[i:i+window] in ENGLISH_WORDS:
            matches+=1
    return matches/(n-window+1)

def vowel_consonant_alt(domain):
    domain=re.sub(r'[^a-z]','',domain.lower())
    count=0
    for i in range(1,len(domain)):
        if (domain[i] in VOWELS and domain[i-1] in CONSONANTS) or (domain[i] in CONSONANTS and domain[i-1] in VOWELS):
            count+=1
    return count

def char_gini(s):
    counts=Counter(s)
    N=sum(counts.values())
    if N==0: return 0
    return 1 - sum((c/N)**2 for c in counts.values())

def kl_divergence(s):
    counts=Counter([c for c in s if c.isalpha()])
    N=sum(counts.values())
    if N==0: return 0
    dist=np.array([counts.get(chr(97+i),0)/N for i in range(26)])
    return entropy(dist, BENIGN_DIST)

def markov_chain_likelihood(domain):
    if len(domain)<2: return 0
    transitions={}
    for i in range(len(domain)-1):
        pair=(domain[i],domain[i+1])
        transitions[pair]=transitions.get(pair,0)+1
    total=sum(transitions.values())
    return sum(math.log((c/total)+1e-6) for c in transitions.values())

def autocorrelation_score(domain):
    if len(domain)<2: return 0
    vals=[ord(c) for c in domain]
    mean=np.mean(vals)
    var=np.var(vals)
    return sum((vals[i]-mean)*(vals[i+1]-mean) for i in range(len(vals)-1))/(var*(len(vals)-1)+1e-6)

def hyphen_word_match_ratio(domain):
    parts = domain.split("-")
    if not parts: return 0
    dict_matches = sum(1 for p in parts if p in ENGLISH_WORDS)
    return dict_matches / len(parts)

def max_cons_cluster(domain):
    return max((len(m.group()) for m in re.finditer(r"[bcdfghjklmnpqrstvwxyz]+", domain.lower())), default=0)

def consonant_count(domain):
    return sum(c in CONSONANTS for c in domain.lower())

def vowel_count(domain):
    return sum(c in VOWELS for c in domain.lower())

def pronouncability_score(domain):
    return vowel_count(domain)/ (consonant_count(domain)+1)

def bigram_score(domain):
    return sum(1 for i in range(len(domain)-1) if domain[i].isalpha() and domain[i+1].isalpha())

def trigram_score(domain):
    return sum(1 for i in range(len(domain)-2) if domain[i].isalpha() and domain[i+2].isalpha())

def ngram_lm_perplexity(domain):
    likelihood = markov_chain_likelihood(domain)
    if len(domain)==0:
        return 0
    return math.exp(-likelihood / len(domain))

def char_freq_deviation(domain):
    counts = list(Counter(domain).values())
    return np.std(counts) if counts else 0

def normal_char_freq_variance(domain):
    counts = list(Counter(domain).values())
    return (np.std(counts)/len(domain)) if domain else 0
def min_levenshtein_to_popular(domain, popular_list=None):
    """
    Compute minimum Levenshtein distance of domain to a list of popular domains.
    """
    if popular_list is None:
        popular_list = ["google.com", "facebook.com", "youtube.com", "amazon.com", "wikipedia.org",
                        "twitter.com", "instagram.com", "linkedin.com", "netflix.com", "yahoo.com"]

    domain = domain.lower()
    min_dist = float("inf")

    for pd in popular_list:
        # Simple Levenshtein via SequenceMatcher ratio
        ratio = difflib.SequenceMatcher(None, domain, pd).ratio()
        dist = max(len(domain), len(pd)) * (1 - ratio)   # convert similarity → distance
        if dist < min_dist:
            min_dist = dist

    return min_dist

# -------------------------
# Main feature extractor
# -------------------------
def extract_features(domain):
    if not isinstance(domain, str): domain = str(domain)
    domain = domain.lower()
    dwc, lwl, lwr = dict_word_features(domain)
    tld = domain.split(".")[-1] if "." in domain else ""

    digits = [c for c in domain if c.isdigit()]
    max_consec_digits = max((len(m.group()) for m in re.finditer(r'\d+', domain)), default=0)
    has_hyphen = 1 if "-" in domain else 0

    feats = {
        # === Core Structural Features ===
        "Length": len(domain),
        "Unique_Chars": len(set(domain)),
        "Uinque_Char_Ratio": len(set(domain))/max(1,len(domain)),
        "Max_Cons_Cluster": max_cons_cluster(domain),
        "Dist_STD": np.std(list(Counter(domain).values())),
        "Char_Gini": char_gini(domain),
        "Char_Freq_Deviation": char_freq_deviation(domain),
        "KL_Divergence": kl_divergence(domain),
        "Compression_Ratio": compress_ratio(domain),
        "Pronounceability": pronouncability_score(domain),
        "Bigram_Score": bigram_score(domain),
        "Trigram_Score": trigram_score(domain),
        "Bigram_Likelihood": bigram_score(domain)/max(1,len(domain)),
        "Markov_Chain_Likelihood": markov_chain_likelihood(domain),
        "Ngram_LM_Perplexity": ngram_lm_perplexity(domain),
        "Renyi_Entropy": renyi_entropy(domain),

        "Keyboard_Distance_Score": keyboard_distance_score(domain),
        "Kolmogorov_Complexity": len(zlib.compress(domain.encode())),
        "Min_Levenshtein_To_Popular": min_levenshtein_to_popular(domain),
        "Sliding_Word_Ratio": sliding_dict_ratio(domain),
        "Normal_Char_Freq_Var": normal_char_freq_variance(domain),
        "Consonant_Count": consonant_count(domain),











        #"Digit_Count": len(digits),
        #"Markov_Cross_Entropy": shannon_entropy(domain) - markov_chain_likelihood(domain),
        #"Digit_Ratio": len(digits)/max(1,len(domain)),
        #"Max_Consec_Digits": max_consec_digits,
        #"Vowel_Count": vowel_count(domain),


        #"Has_Hyphen": has_hyphen,
        #"Subdomain_Depth": domain.count("."),
        #
        # === Statistical / Info-Theoretic Features ===
        #"Entropy": shannon_entropy(domain),
        #"Normalized_Entropy": normalized_entropy(domain),
        #"Vowel_Entropy": shannon_entropy("".join([c for c in domain if c in VOWELS])),
        #"Consonant_Entropy": shannon_entropy("".join([c for c in domain if c in CONSONANTS])),

        # === Dictionary & Word Features ===
        #"Word_Match_Ratio": dwc/max(1,len(domain.split("."))),

        #"Dict_Word_Count": dwc,
        #"Longest_Word_Len": lwl,
        #"Longest_Word_Ratio": lwr,
        #"Min_Levenshtein_Top": min_levenshtein_to_popular(domain),
        # === Pronounceability / Lexical Features ===

        #"Vowel_Consonant_Alt": vowel_consonant_alt(domain),
        #"Lexical_Complexity": shannon_entropy(domain)*len(set(domain))/max(1,len(domain)),
        # === N-gram / Language Model ===

        # === TLD / Popularity ===
        #"TLD_Common": 1 if f".{tld}" in POPULAR_TLDS else 0,
        #"TLD_Bad_Score": 1 if f".{tld}" in BAD_TLDS else 0,
        #"Popular_Domain": 1 if domain in POPULAR_DOMAINS else 0,
        #"Alexa_popularity_bucket": -1,  # placeholder
        #"TLD_Length": len(tld),
        #"TLD_Rarity_Score": 0 if f".{tld}" in POPULAR_TLDS else 1,
        # === Structural / Pattern Features ===
        #"Repeat_Chars": max((len(m.group()) for m in re.finditer(r'(.)\1+', domain)), default=0),
        #"Max_Repeated_Char_Run": max((len(m.group()) for m in re.finditer(r'(.)\1+', domain)), default=0),
        #"Digit_Letter_Alt_Count": sum((domain[i].isdigit() != domain[i+1].isdigit()) for i in range(len(domain)-1)),
        #"Palindromic_Substring_Ratio": sum(domain[i:j]==domain[i:j][::-1] for i in range(len(domain)) for j in range(i+2,len(domain)+1))/max(1,len(domain)),
        #"First_Last_Char_Type": int(domain[0].isalpha()) + int(domain[-1].isalpha()),
        #"Permutation_Entropy": shannon_entropy(domain)/max(1,math.log2(len(domain))),
        #"Word_Boundary_Count": domain.count("-"),
        #"Syllable_Count": sum(domain.count(v) for v in VOWELS),
        #"Hyphen_Placement_Score": 1 if domain.startswith("-") or domain.endswith("-") else 0,

        #"RunLength_Entropy": shannon_entropy("".join([str(len(m.group())) for m in re.finditer(r'(.)\1*', domain)])),

        #"Autocorrelation_Score": autocorrelation_score(domain),
        #"Hurst_Exponent": 0.5,  # placeholder
        #"Hyphen_Word_Match_Ratio": hyphen_word_match_ratio(domain),

    }
    return feats



def compute_features_for_dataset(df, domain_col="domain"):
    feature_rows = []
    for domain in df[domain_col]:
        feats = extract_features(domain.lower())
        feature_rows.append(feats)
    if feature_rows:
      # Create DataFrame from list of dictionaries
      feature_df = pd.DataFrame(feature_rows)
      # Ensure columns match the keys from extract_features
      feature_df = feature_df[list(feature_rows[0].keys())]
      return feature_df
    else:
      return pd.DataFrame()


# =========================
# Training Phase
# =========================
def train_model(train_file, label_col="label", model_type="xgboost", save_model="dga_model.pkl"):
    # Load training dataset (already has features + labels)
    df = pd.read_csv(train_file)

    # Drop label + domain if exists
    drop_cols = [label_col]
    if "domain" in df.columns:
        drop_cols.append("domain")

    X = df.drop(columns=drop_cols)
    y = df[label_col]

    # Ensure all features are numeric
    X = X.apply(pd.to_numeric, errors="coerce").fillna(0)

    X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

    if model_type == "xgboost":
        model = XGBClassifier(use_label_encoder=False, eval_metric="logloss")
    else:
        model = RandomForestClassifier(n_estimators=200, random_state=42)

    model.fit(X_train, y_train)
    preds = model.predict(X_val)

    print("Validation Accuracy:", accuracy_score(y_val, preds))
    print(classification_report(y_val, preds))

    joblib.dump(model, save_model)
    print(f"✅ Model saved to {save_model}")
    return model


# =========================
# Prediction Phase
# =========================
def predict_new(test_file, model_file="dga_model.pkl", output_file="predictions_ndga.csv"):
    # Load test dataset
    df_test = pd.read_csv(test_file)

    # Detect which column to use for domains
    if "domain" in df_test.columns:
        domain_col = "domain"
    else:
        # Assume the first column holds the domain strings
        domain_col = df_test.columns[0]

    # Compute features
    X_test = compute_features_for_dataset(df_test, domain_col=domain_col)

    # Ensure numeric dtype
    X_test = X_test.apply(pd.to_numeric, errors="coerce").fillna(0)

    # Load trained model
    model = joblib.load(model_file)

    # Predict
    preds = model.predict(X_test)

    # Attach predictions alongside original domains
    df_out = df_test.copy()
    df_out["Prediction"] = preds

    df_out.to_csv(output_file, index=False)
    print(f"✅ Predictions saved to {output_file}")
    return df_out

# =========================
# Example Usage
# =========================
# Train the model
model = train_model("combined.csv", label_col="label", model_type="xgboost")

# Predict on new domains
results = predict_new("dataset_ndga.csv", model_file="dga_model.pkl")
print(results.head())


[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Package words is already up-to-date!
/usr/local/lib/python3.12/dist-packages/xgboost/training.py:183: UserWarning: [18:23:57] WARNING: /workspace/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


Validation Accuracy: 0.7009483143109915
              precision    recall  f1-score   support

           0       0.69      0.73      0.71    136596
           1       0.71      0.67      0.69    136942

    accuracy                           0.70    273538
   macro avg       0.70      0.70      0.70    273538
weighted avg       0.70      0.70      0.70    273538

✅ Model saved to dga_model.pkl



KeyboardInterrupt



In [ ]:
import pandas as pd

from google.colab import files
pd.download("combined.csv")

In [ ]:
import pandas as pd
#Min_Levenshtein_to_Popular"
def count_zero_one(df, column_name):
    """
    Count how many entries are 0 or 1 in a specific column.

    Parameters:
    -----------
    df : pd.DataFrame
        Input dataset
    column_name : str
        Column to check

    Returns:
    --------
    dict : counts of 0 and 1
    """
    if column_name not in df.columns:
        raise ValueError(f"Column '{column_name}' not found in dataset")

    counts = {
        "count_0": (df[column_name] == 0).sum(),
        "count_1": (df[column_name] == 1).sum()
    }
    return counts


# ==== Example Usage ====
df = pd.read_csv("predictions_dga.csv")

# Replace "Label" with your column name
result = count_zero_one(df, "Prediction")

print(f"Number of 0s: {result['count_0']}")
print(f"Number of 1s: {result['count_1']}")



##DGA-86%
## Benign 87%

##DGA-89%
## Benign 86%

In [ ]:
import pandas as pd
import numpy as np
import re
import math
import joblib
from collections import Counter
from xgboost import XGBClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score
from difflib import SequenceMatcher # Import SequenceMatcher

# =========================
# Feature Extraction Utils
# =========================

try:
    from nltk.corpus import words
    from nltk import download
    download('words')
    ENGLISH_WORDS = set(w.lower() for w in words.words())
except:
    ENGLISH_WORDS = {"test", "shop", "mail", "home", "secure", "login"}  # fallback small dict

VOWELS = set("aeiou")
CONSONANTS = set("bcdfghjklmnpqrstvwxyz")
POPULAR_TLDS = {".com", ".org", ".net", ".info", ".edu", ".gov"}
BAD_TLDS = {".xyz", ".top", ".club", ".work", ".click"}

# Load top domains (free list like Tranco or Cisco Umbrella top domains)
try:
    with open("top1k_domains.txt") as f:
        popular_domains = [line.strip().lower() for line in f.readlines()]
except FileNotFoundError:
    popular_domains = ["google.com","facebook.com","youtube.com","amazon.com","wikipedia.org"]  # fallback demo list


def shannon_entropy(s):
    if not s:
        return 0
    probs = [s.count(c) / len(s) for c in set(s)]
    return -sum(p * math.log2(p) for p in probs)

def char_gini(domain: str) -> float:
    """Compute Gini index of character distribution in the domain"""
    if not domain:
        return 0.0
    counts = np.array([domain.count(c) for c in set(domain)])
    probs = counts / counts.sum()
    return 1.0 - np.sum(probs ** 2)

def get_tld(domain: str) -> str:
    """Extracts the top-level domain (TLD) from a domain string."""
    parts = domain.split(".")
    if len(parts) > 1:
        return "." + parts[-1]  # include dot for matching with POPULAR_TLDS/BAD_TLDS
    return ""


def digit_count(domain):
    return sum(c.isdigit() for c in domain)

def vowel_count(domain):
    return sum(c in VOWELS for c in domain)

def consonant_count(domain):
    return sum(c in CONSONANTS for c in domain)

def unique_chars(domain):
    return len(set(domain))

def has_hyphen(domain):
    return 1 if "-" in domain else 0

def subdomain_depth(domain):
    return domain.count(".")  # depth based on dots

def max_consec_digits(domain):
    return max((len(m.group()) for m in re.finditer(r"\d+", domain)), default=0)

def max_cons_cluster(domain):
    return max((len(m.group()) for m in re.finditer(r"[bcdfghjklmnpqrstvwxyz]+", domain)), default=0)

def hyphen_word_match_ratio(domain: str) -> float:
    """Ratio of hyphen-separated parts that are valid dictionary words."""
    parts = domain.split("-")
    if not parts:
        return 0.0
    dict_matches = sum(1 for p in parts if p in ENGLISH_WORDS)
    return dict_matches / len(parts)

# === Extra Feature Functions ===
def renyi_entropy(s, alpha=2):
    """Rényi entropy of order alpha (default α=2)."""
    if not s:
        return 0
    counts = Counter(s)
    probs = np.array(list(counts.values())) / len(s)
    if alpha == 1:
        return -np.sum(probs * np.log2(probs))  # Shannon
    return 1 / (1 - alpha) * np.log2(np.sum(probs ** alpha))

# Load top domains (free list like Tranco or Cisco Umbrella top domains)
try:
    with open("top1k_domains.txt") as f:
        popular_domains = [line.strip().lower() for line in f.readlines()]
except FileNotFoundError:
    popular_domains = ["google.com","facebook.com","youtube.com","amazon.com","wikipedia.org"]  # fallback demo list

def min_levenshtein_to_popular(domain, topN=500):
    """Compute minimum normalized Levenshtein distance to topN popular domains."""
    domain = domain.lower()
    min_dist = 1.0
    for pd in popular_domains[:topN]:
        ratio = SequenceMatcher(None, domain, pd).ratio()
        dist = 1 - ratio  # 0 = identical, 1 = very different
        if dist < min_dist:
            min_dist = dist
    return min_dist

# Keyboard adjacency mapping
KEYBOARD_NEIGHBORS = {
    'q': "was", 'w': "qase", 'e': "wsdr", 'r': "edft", 't': "rfgy",
    'y': "tghu", 'u': "yhj", 'i': "ujk", 'o': "ikl", 'p': "ol",
    'a': "qwsz", 's': "qwedxza", 'd': "erfcxs", 'f': "rtgvcd", 'g': "tyhbvf",
    'h': "yujnbg", 'j': "uikmnh", 'k': "iolmj", 'l': "opk",
    'z': "asx", 'x': "zsdc", 'c': "xdfv", 'v': "cfgb", 'b': "vghn",
    'n': "bhjm", 'm': "njk"
}

def keyboard_distance_score(domain):
    """Average keyboard adjacency match score between consecutive characters."""
    domain = re.sub(r'[^a-z]', '', domain.lower())
    if len(domain) < 2:
        return 0
    score = 0
    for i in range(len(domain)-1):
        if domain[i+1] in KEYBOARD_NEIGHBORS.get(domain[i], ""):
            score += 1
    return score / (len(domain)-1)


def extract_features(domain):
    if not isinstance(domain, str):
        domain = str(domain)
    domain = domain.lower()
    name, _, tld = domain.rpartition(".")
    if not name:
        name = domain

    feats = {
         #"Min_Levenshtein_to_Popular": min_levenshtein_to_popular(domain),
         "Keyboard_Distance_Score": keyboard_distance_score(domain)
    }
    return feats


def compute_features_for_dataset(df, domain_col="domain"):
    feature_rows = []
    for domain in df[domain_col]:
        feats = extract_features(domain.lower())
        feature_rows.append(feats)
    if feature_rows:
      # Create DataFrame from list of dictionaries
      feature_df = pd.DataFrame(feature_rows)
      # Ensure columns match the keys from extract_features
      feature_df = feature_df[list(feature_rows[0].keys())]
      return feature_df
    else:
      return pd.DataFrame()


# =========================
# Training Phase
# =========================
def train_model(train_file, label_col="label", model_type="xgboost", save_model="dga_model.pkl"):
    # Load training dataset (already has features + labels)
    df = pd.read_csv(train_file)

    # Drop label + domain if exists
    drop_cols = [label_col]
    if "domain" in df.columns:
        drop_cols.append("domain")

    X = df.drop(columns=drop_cols)
    y = df[label_col]

    # Ensure all features are numeric
    X = X.apply(pd.to_numeric, errors="coerce").fillna(0)

    X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

    if model_type == "xgboost":
        model = XGBClassifier(use_label_encoder=False, eval_metric="logloss")
    else:
        model = RandomForestClassifier(n_estimators=200, random_state=42)

    model.fit(X_train, y_train)
    preds = model.predict(X_val)

    print("Validation Accuracy:", accuracy_score(y_val, preds))
    print(classification_report(y_val, preds))

    joblib.dump(model, save_model)
    print(f"✅ Model saved to {save_model}")
    return model


# =========================
# Prediction Phase
# =========================
def predict_new(test_file, model_file="dga_model.pkl", output_file="predictions_ndga.csv"):
    # Load test dataset
    df_test = pd.read_csv(test_file)

    # Detect which column to use for domains
    if "domain" in df_test.columns:
        domain_col = "domain"
    else:
        # Assume the first column holds the domain strings
        domain_col = df_test.columns[0]

    # Compute features
    X_test = compute_features_for_dataset(df_test, domain_col=domain_col)

    # Ensure numeric dtype
    X_test = X_test.apply(pd.to_numeric, errors="coerce").fillna(0)

    # Load trained model
    model = joblib.load(model_file)

    # Predict
    preds = model.predict(X_test)

    # Attach predictions alongside original domains
    df_out = df_test.copy()
    df_out["Prediction"] = preds

    df_out.to_csv(output_file, index=False)
    print(f"✅ Predictions saved to {output_file}")
    return df_out

# =========================
# Example Usage
# =========================
# Train the model
model = train_model("combined.csv", label_col="label", model_type="xgboost")

# Predict on new domains
results = predict_new("dataset_ndga.csv", model_file="dga_model.pkl")
print(results.head())


In [ ]:
import pandas as pd

def count_zero_one(df, column_name):
    """
    Count how many entries are 0 or 1 in a specific column.

    Parameters:
    -----------
    df : pd.DataFrame
        Input dataset
    column_name : str
        Column to check

    Returns:
    --------
    dict : counts of 0 and 1
    """
    if column_name not in df.columns:
        raise ValueError(f"Column '{column_name}' not found in dataset")

    counts = {
        "count_0": (df[column_name] == 0).sum(),
        "count_1": (df[column_name] == 1).sum()
    }
    return counts


# ==== Example Usage ====
df = pd.read_csv("predictions_ndga.csv")

# Replace "Label" with your column name
result = count_zero_one(df, "Prediction")

print(f"Number of 0s: {result['count_0']}")
print(f"Number of 1s: {result['count_1']}")
 ##dga 79
 ## 68


In [ ]:
 # 0 ---> 84/75



In [ ]:
#all dga 90%
#benign 86%
#-2:compression ratio
#:Bigram score
# dga 89
#BENIGN: 86

In [ ]:
# =========================
# DGA Feature Extraction + Model Training Pipeline
# =========================

import pandas as pd
import numpy as np
import re
import math
import zlib
import string
from collections import Counter
from difflib import SequenceMatcher
from xgboost import XGBClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score
from nltk.corpus import words
from nltk import download
from scipy.stats import entropy

# -------------------------
# Download English words
# -------------------------
try:
    download('words')
    ENGLISH_WORDS = set(w.lower() for w in words.words())
except:
    ENGLISH_WORDS = {"test","shop","login","home","mail","secure"}  # fallback

# -------------------------
# Constants
# -------------------------
VOWELS = set("aeiou")
CONSONANTS = set(string.ascii_lowercase) - VOWELS
POPULAR_TLDS = {".com", ".org", ".net", ".info", ".edu", ".gov"}
BAD_TLDS = {".xyz", ".top", ".club", ".work", ".click"}
BENIGN_DIST = np.ones(26)/26  # uniform for KL divergence

# -------------------------
# Load top domains
# -------------------------
try:
    with open("top1k_domains.txt") as f:
        POPULAR_DOMAINS = [line.strip().lower() for line in f.readlines()]
except FileNotFoundError:
    POPULAR_DOMAINS = ["google.com","facebook.com","youtube.com","amazon.com","wikipedia.org"]

# -------------------------
# Feature Functions
# -------------------------
def shannon_entropy(s):
    if not s: return 0
    p, _ = np.histogram(list(s), bins=range(257), density=True)
    p = p[p>0]
    return -np.sum(p*np.log2(p))

def normalized_entropy(s):
    if not s: return 0
    return shannon_entropy(s)/math.log2(len(set(s))+1)

def compress_ratio(s):
    if not s: return 0
    return len(zlib.compress(s.encode()))/max(1,len(s))

def renyi_entropy(s, alpha=2):
    if not s: return 0
    counts = Counter(s)
    probs = np.array(list(counts.values())) / len(s)
    if alpha == 1:
        return -np.sum(probs*np.log2(probs))
    return 1/(1-alpha) * np.log2(np.sum(probs**alpha))

def min_levenshtein_to_popular(domain, topN=500):
    domain = domain.lower()
    min_dist = 1.0
    for pd in POPULAR_DOMAINS[:topN]:
        ratio = SequenceMatcher(None, domain, pd).ratio()
        dist = 1 - ratio
        if dist < min_dist:
            min_dist = dist
    return min_dist

KEYBOARD_NEIGHBORS = {
    'q':"was",'w':"qase",'e':"wsdr",'r':"edft",'t':"rfgy",
    'y':"tghu",'u':"yhj",'i':"ujk",'o':"ikl",'p':"ol",
    'a':"qwsz",'s':"qwedxza",'d':"erfcxs",'f':"rtgvcd",'g':"tyhbvf",
    'h':"yujnbg",'j':"uikmnh",'k':"iolmj",'l':"opk",
    'z':"asx",'x':"zsdc",'c':"xdfv",'v':"cfgb",'b':"vghn",
    'n':"bhjm",'m':"njk"
}

def keyboard_distance_score(domain):
    domain = re.sub(r'[^a-z]','',domain.lower())
    if len(domain)<2: return 0
    score=0
    for i in range(len(domain)-1):
        if domain[i+1] in KEYBOARD_NEIGHBORS.get(domain[i],""):
            score+=1
    return score/(len(domain)-1)

def dict_word_features(domain):
    matches = [w for w in ENGLISH_WORDS if w in domain]
    longest = max((len(w) for w in matches), default=0)
    return len(matches), longest, longest/max(1,len(domain))

def sliding_dict_ratio(domain, window=4):
    n=len(domain)
    if n<window: return 0
    matches=0
    for i in range(n-window+1):
        if domain[i:i+window] in ENGLISH_WORDS:
            matches+=1
    return matches/(n-window+1)

def vowel_consonant_alt(domain):
    domain=re.sub(r'[^a-z]','',domain.lower())
    count=0
    for i in range(1,len(domain)):
        if (domain[i] in VOWELS and domain[i-1] in CONSONANTS) or (domain[i] in CONSONANTS and domain[i-1] in VOWELS):
            count+=1
    return count

def char_gini(s):
    counts=Counter(s)
    N=sum(counts.values())
    if N==0: return 0
    return 1 - sum((c/N)**2 for c in counts.values())

def kl_divergence(s):
    counts=Counter([c for c in s if c.isalpha()])
    N=sum(counts.values())
    if N==0: return 0
    dist=np.array([counts.get(chr(97+i),0)/N for i in range(26)])
    return entropy(dist, BENIGN_DIST)

def markov_chain_likelihood(domain):
    if len(domain)<2: return 0
    transitions={}
    for i in range(len(domain)-1):
        pair=(domain[i],domain[i+1])
        transitions[pair]=transitions.get(pair,0)+1
    total=sum(transitions.values())
    return sum(math.log((c/total)+1e-6) for c in transitions.values())

def autocorrelation_score(domain):
    if len(domain)<2: return 0
    vals=[ord(c) for c in domain]
    mean=np.mean(vals)
    var=np.var(vals)
    return sum((vals[i]-mean)*(vals[i+1]-mean) for i in range(len(vals)-1))/(var*(len(vals)-1)+1e-6)

def hyphen_word_match_ratio(domain):
    parts = domain.split("-")
    if not parts: return 0
    dict_matches = sum(1 for p in parts if p in ENGLISH_WORDS)
    return dict_matches / len(parts)

def max_cons_cluster(domain):
    return max((len(m.group()) for m in re.finditer(r"[bcdfghjklmnpqrstvwxyz]+", domain.lower())), default=0)

def consonant_count(domain):
    return sum(c in CONSONANTS for c in domain.lower())

def vowel_count(domain):
    return sum(c in VOWELS for c in domain.lower())

def pronouncability_score(domain):
    return vowel_count(domain)/ (consonant_count(domain)+1)

def bigram_score(domain):
    return sum(1 for i in range(len(domain)-1) if domain[i].isalpha() and domain[i+1].isalpha())

def trigram_score(domain):
    return sum(1 for i in range(len(domain)-2) if domain[i].isalpha() and domain[i+2].isalpha())

def ngram_lm_perplexity(domain):
    likelihood = markov_chain_likelihood(domain)
    if len(domain)==0:
        return 0
    return math.exp(-likelihood / len(domain))

def char_freq_deviation(domain):
    counts = list(Counter(domain).values())
    return np.std(counts) if counts else 0

def normal_char_freq_variance(domain):
    counts = list(Counter(domain).values())
    return (np.std(counts)/len(domain)) if domain else 0

# -------------------------
# Main feature extractor

def extract_features(domain):
    if not isinstance(domain, str): domain = str(domain)
    domain = domain.lower()
    dwc, lwl, lwr = dict_word_features(domain)
    tld = domain.split(".")[-1] if "." in domain else ""

    digits = [c for c in domain if c.isdigit()]
    max_consec_digits = max((len(m.group()) for m in re.finditer(r'\d+', domain)), default=0)
    has_hyphen = 1 if "-" in domain else 0

    feats = {
        # === Core Structural Features ===
        "Length": len(domain),
        "Digit_Count": len(digits),
        "Digit_Ratio": len(digits)/max(1,len(domain)),
        "Max_Consec_Digits": max_consec_digits,
        "Vowel_Count": vowel_count(domain),
        "Consonant_Count": consonant_count(domain),
        "Unique_Chars": len(set(domain)),
        "Has_Hyphen": has_hyphen,
        "Subdomain_Depth": domain.count("."),
        "Max_Cons_Cluster": max_cons_cluster(domain),
        # === Statistical / Info-Theoretic Features ===
        "Entropy": shannon_entropy(domain),
        "Normalized_Entropy": normalized_entropy(domain),
        "Vowel_Entropy": shannon_entropy("".join([c for c in domain if c in VOWELS])),
        "Consonant_Entropy": shannon_entropy("".join([c for c in domain if c in CONSONANTS])),
        "Dist_STD": np.std(list(Counter(domain).values())),
        "Char_Gini": char_gini(domain),
        "Char_Freq_Deviation": char_freq_deviation(domain),
        "KL_Divergence": kl_divergence(domain),
        "Compression_Ratio": compress_ratio(domain),
        # === Dictionary & Word Features ===
        "Word_Match_Ratio": dwc/max(1,len(domain.split("."))),
        "Sliding_Word_Ratio": sliding_dict_ratio(domain),
        "Dict_Word_Count": dwc,
        "Longest_Word_Len": lwl,
        "Longest_Word_Ratio": lwr,
        "Min_Levenshtein_Top": min_levenshtein_to_popular(domain),
        # === Pronounceability / Lexical Features ===
        "Pronounceability": pronouncability_score(domain),
        "Vowel_Consonant_Alt": vowel_consonant_alt(domain),
        "Lexical_Complexity": shannon_entropy(domain)*len(set(domain))/max(1,len(domain)),
        # === N-gram / Language Model ===
        "Bigram_Score": bigram_score(domain),
        "Trigram_Score": trigram_score(domain),
        "Bigram_Likelihood": bigram_score(domain)/max(1,len(domain)),
        "Char_Markov_Likelihood": markov_chain_likelihood(domain),
        "Ngram_LM_Perplexity": ngram_lm_perplexity(domain),
        # === TLD / Popularity ===
        "TLD_Common": 1 if f".{tld}" in POPULAR_TLDS else 0,
        "TLD_Bad_Score": 1 if f".{tld}" in BAD_TLDS else 0,
        "Popular_Domain": 1 if domain in POPULAR_DOMAINS else 0,
        "Alexa_popularity_bucket": -1,  # placeholder
        "TLD_Length": len(tld),
        "TLD_Rarity_Score": 0 if f".{tld}" in POPULAR_TLDS else 1,
        # === Structural / Pattern Features ===
        "Repeat_Chars": max((len(m.group()) for m in re.finditer(r'(.)\1+', domain)), default=0),
        "Max_Repeated_Char_Run": max((len(m.group()) for m in re.finditer(r'(.)\1+', domain)), default=0),
        "Digit_Letter_Alt_Count": sum((domain[i].isdigit() != domain[i+1].isdigit()) for i in range(len(domain)-1)),
        "Palindromic_Substring_Ratio": sum(domain[i:j]==domain[i:j][::-1] for i in range(len(domain)) for j in range(i+2,len(domain)+1))/max(1,len(domain)),
        "First_Last_Char_Type": int(domain[0].isalpha()) + int(domain[-1].isalpha()),
        "Permutation_Entropy": shannon_entropy(domain)/max(1,math.log2(len(domain))),
        "Word_Boundary_Count": domain.count("-"),
        "Syllable_Count": sum(domain.count(v) for v in VOWELS),
        "Hyphen_Placement_Score": 1 if domain.startswith("-") or domain.endswith("-") else 0,
        "Renyi_Entropy": renyi_entropy(domain),
        "Markov_Cross_Entropy": shannon_entropy(domain) - markov_chain_likelihood(domain),
        "RunLength_Entropy": shannon_entropy("".join([str(len(m.group())) for m in re.finditer(r'(.)\1*', domain)])),
        "Normalized_Char_Freq_Var": normal_char_freq_variance(domain),
        "Autocorrelation_Score": autocorrelation_score(domain),
        "Hurst_Exponent": 0.5,  # placeholder
        "Hyphen_Word_Match_Ratio": hyphen_word_match_ratio(domain),
        "Keyboard_Distance_Score": keyboard_distance_score(domain)
    }
    return feats


# -------------------------
# Dataset Feature Computation
# -------------------------
def compute_features_for_dataset(df, domain_col="domain"):
    return pd.DataFrame([extract_features(d) for d in df[domain_col]])

# -------------------------
# Train Model
# -------------------------
def train_model(train_file, label_col="label", model_type="xgboost", save_model="dga_model.pkl"):
    df = pd.read_csv(train_file)
    X = compute_features_for_dataset(df)
    y = df[label_col]

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    if model_type=="xgboost":
        model = XGBClassifier(use_label_encoder=False, eval_metric='logloss', n_jobs=-1)
    else:
        model = RandomForestClassifier(n_estimators=200, n_jobs=-1)

    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)

    print("Accuracy:", accuracy_score(y_test, y_pred))
    print("Classification Report:\n", classification_report(y_test, y_pred))

    joblib.dump(model, save_model)
    print(f"✅ Model saved to {save_model}")
    return model
